# setup

In [81]:
%%capture
# %pip install sae-lens
%pip install umap-learn # cant use with transformelns?

In [ ]:
# import pdb

In [2]:
import pickle
import numpy as np

import torch
import matplotlib.pyplot as plt

# from sae_lens import SAE

from torch import nn, Tensor
# from jaxtyping import Float, Int
from typing import Optional, Callable, Union, List, Tuple

# from datasets import load_dataset

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

## corr fns

In [ ]:
def batched_correlation(reshaped_activations_A, reshaped_activations_B, batch_size=100):
    # Ensure tensors are on GPU
    if torch.cuda.is_available():
        reshaped_activations_A = reshaped_activations_A.to('cuda')
        reshaped_activations_B = reshaped_activations_B.to('cuda')

    # Normalize columns of A
    mean_A = reshaped_activations_A.mean(dim=0, keepdim=True)
    std_A = reshaped_activations_A.std(dim=0, keepdim=True)
    normalized_A = (reshaped_activations_A - mean_A) / (std_A + 1e-8)  # Avoid division by zero

    # Normalize columns of B
    mean_B = reshaped_activations_B.mean(dim=0, keepdim=True)
    std_B = reshaped_activations_B.std(dim=0, keepdim=True)
    normalized_B = (reshaped_activations_B - mean_B) / (std_B + 1e-8)  # Avoid division by zero

    num_batches = (normalized_B.shape[1] + batch_size - 1) // batch_size
    max_values = []
    max_indices = []

    for batch in range(num_batches):
        start = batch * batch_size
        end = min(start + batch_size, normalized_B.shape[1])
        batch_corr_matrix = torch.matmul(normalized_A.t(), normalized_B[:, start:end]) / normalized_A.shape[0]
        max_val, max_idx = batch_corr_matrix.max(dim=0)
        max_values.append(max_val)
        # max_indices.append(max_idx + start)  # Adjust indices for the batch offset
        max_indices.append(max_idx)  # Adjust indices for the batch offset

        del batch_corr_matrix
        torch.cuda.empty_cache()

    return torch.cat(max_indices), torch.cat(max_values)

## sim fns

In [ ]:
import functools
from typing import Any, Callable, Dict, List, Tuple, Union

import numpy as np
import numpy.typing as npt
import torch


def to_numpy_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[npt.NDArray]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> npt.NDArray:
        return x if isinstance(x, np.ndarray) else x.numpy()

    return list(map(convert, args))


def to_torch_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[torch.Tensor]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
        return x if isinstance(x, torch.Tensor) else torch.from_numpy(x)

    return list(map(convert, args))


def adjust_dimensionality(
    R: npt.NDArray, Rp: npt.NDArray, strategy="zero_pad"
) -> Tuple[npt.NDArray, npt.NDArray]:
    D = R.shape[1]
    Dp = Rp.shape[1]
    if strategy == "zero_pad":
        if D - Dp == 0:
            return R, Rp
        elif D - Dp > 0:
            return R, np.concatenate((Rp, np.zeros((Rp.shape[0], D - Dp))), axis=1)
        else:
            return np.concatenate((R, np.zeros((R.shape[0], Dp - D))), axis=1), Rp
    else:
        raise NotImplementedError()


def center_columns(R: npt.NDArray) -> npt.NDArray:
    return R - R.mean(axis=0)[None, :]


def normalize_matrix_norm(R: npt.NDArray) -> npt.NDArray:
    return R / np.linalg.norm(R, ord="fro")


def sim_random_baseline(
    rep1: torch.Tensor, rep2: torch.Tensor, sim_func: Callable, n_permutations: int = 10
) -> Dict[str, Any]:
    torch.manual_seed(1234)
    scores = []
    for _ in range(n_permutations):
        perm = torch.randperm(rep1.size(0))

        score = sim_func(rep1[perm, :], rep2)
        score = score if isinstance(score, float) else score["score"]

        scores.append(score)

    return {"baseline_scores": np.array(scores)}


class Pipeline:
    def __init__(
        self,
        preprocess_funcs: List[Callable[[npt.NDArray], npt.NDArray]],
        similarity_func: Callable[[npt.NDArray, npt.NDArray], Dict[str, Any]],
    ) -> None:
        self.preprocess_funcs = preprocess_funcs
        self.similarity_func = similarity_func

    def __call__(self, R: npt.NDArray, Rp: npt.NDArray) -> Dict[str, Any]:
        for preprocess_func in self.preprocess_funcs:
            R = preprocess_func(R)
            Rp = preprocess_func(Rp)
        return self.similarity_func(R, Rp)

    def __str__(self) -> str:
        def func_name(func: Callable) -> str:
            return (
                func.__name__
                if not isinstance(func, functools.partial)
                else func.func.__name__
            )

        def partial_keywords(func: Callable) -> str:
            if not isinstance(func, functools.partial):
                return ""
            else:
                return str(func.keywords)

        return (
            "Pipeline("
            + (
                "+".join(map(func_name, self.preprocess_funcs))
                + "+"
                + func_name(self.similarity_func)
                + partial_keywords(self.similarity_func)
            )
            + ")"
        )

In [ ]:
from typing import List, Set, Union

import numpy as np
import numpy.typing as npt
import sklearn.neighbors
import torch

# from llmcomp.measures.utils import to_numpy_if_needed


def _jac_sim_i(idx_R: Set[int], idx_Rp: Set[int]) -> float:
    return len(idx_R.intersection(idx_Rp)) / len(idx_R.union(idx_Rp))


def jaccard_similarity(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    k: int = 10,
    inner: str = "cosine",
    n_jobs: int = 8,
) -> float:
    R, Rp = to_numpy_if_needed(R, Rp)

    indices_R = nn_array_to_setlist(top_k_neighbors(R, k, inner, n_jobs))
    indices_Rp = nn_array_to_setlist(top_k_neighbors(Rp, k, inner, n_jobs))

    return float(
        np.mean(
            [_jac_sim_i(idx_R, idx_Rp) for idx_R, idx_Rp in zip(indices_R, indices_Rp)]
        )
    )


def top_k_neighbors(
    R: npt.NDArray,
    k: int,
    inner: str,
    n_jobs: int,
) -> npt.NDArray:
    # k+1 nearest neighbors, because we pass in all the data, which means that a point
    # will be the nearest neighbor to itself. We remove this point from the results and
    # report only the k nearest neighbors distinct from the point itself.
    nns = sklearn.neighbors.NearestNeighbors(
        n_neighbors=k + 1, metric=inner, n_jobs=n_jobs
    )
    nns.fit(R)
    _, nns = nns.kneighbors(R)
    return nns[:, 1:]


def nn_array_to_setlist(nn: npt.NDArray) -> List[Set[int]]:
    return [set(idx) for idx in nn]

In [ ]:
import functools
import logging
from abc import ABC
from abc import abstractmethod
from dataclasses import dataclass
from dataclasses import field
from typing import Any
from typing import Callable
from typing import get_args
from typing import List
from typing import Literal
from typing import Optional
from typing import Protocol
from typing import Tuple
from typing import Union

import numpy as np
import numpy.typing as npt
import torch
from einops import rearrange
# from loguru import logger

log = logging.getLogger(__name__)


SHAPE_TYPE = Literal["nd", "ntd", "nchw"]

ND_SHAPE, NTD_SHAPE, NCHW_SHAPE = get_args(SHAPE_TYPE)[0], get_args(SHAPE_TYPE)[1], get_args(SHAPE_TYPE)[2]


class SimilarityFunction(Protocol):
    def __call__(  # noqa: E704
        self,
        R: torch.Tensor | npt.NDArray,
        Rp: torch.Tensor | npt.NDArray,
        shape: SHAPE_TYPE,
    ) -> float: ...


class RSMSimilarityFunction(Protocol):
    def __call__(  # noqa: E704
        self, R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray, shape: SHAPE_TYPE, n_jobs: int
    ) -> float: ...


@dataclass
class BaseSimilarityMeasure(ABC):
    larger_is_more_similar: bool
    is_symmetric: bool

    is_metric: bool | None = None
    invariant_to_affine: bool | None = None
    invariant_to_invertible_linear: bool | None = None
    invariant_to_ortho: bool | None = None
    invariant_to_permutation: bool | None = None
    invariant_to_isotropic_scaling: bool | None = None
    invariant_to_translation: bool | None = None
    name: str = field(init=False)

    def __post_init__(self):
        self.name = self.__class__.__name__

    @abstractmethod
    def __call__(self, *args: Any, **kwds: Any) -> Any:
        raise NotImplementedError


class FunctionalSimilarityMeasure(BaseSimilarityMeasure):
    @abstractmethod
    def __call__(self, output_a: torch.Tensor | npt.NDArray, output_b: torch.Tensor | npt.NDArray) -> float:
        raise NotImplementedError


@dataclass(kw_only=True)
class RepresentationalSimilarityMeasure(BaseSimilarityMeasure):
    sim_func: SimilarityFunction

    def __call__(
        self,
        R: torch.Tensor | npt.NDArray,
        Rp: torch.Tensor | npt.NDArray,
        shape: SHAPE_TYPE,
    ) -> float:
        return self.sim_func(R, Rp, shape)


class RSMSimilarityMeasure(RepresentationalSimilarityMeasure):
    sim_func: RSMSimilarityFunction

    @staticmethod
    def estimate_good_number_of_jobs(R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray) -> int:
        # RSMs in are NxN (or DxD) so the number of jobs should roughly scale quadratically with increase in N (or D).
        # False! As long as sklearn-native metrics are used, they will use parallel implementations regardless of job
        # count. Each job would spawn their own threads, which leads to oversubscription of cores and thus slowdown.
        # This seems to be not fully correct (n_jobs=2 seems to actually use two cores), but using n_jobs=1 seems the
        # fastest.
        return 1

    def __call__(
        self,
        R: torch.Tensor | npt.NDArray,
        Rp: torch.Tensor | npt.NDArray,
        shape: SHAPE_TYPE,
        n_jobs: Optional[int] = None,
    ) -> float:
        if n_jobs is None:
            n_jobs = self.estimate_good_number_of_jobs(R, Rp)
        return self.sim_func(R, Rp, shape, n_jobs=n_jobs)


def to_numpy_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[npt.NDArray]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> npt.NDArray:
        return x if isinstance(x, np.ndarray) else x.numpy()

    return list(map(convert, args))


def to_torch_if_needed(*args: Union[torch.Tensor, npt.NDArray]) -> List[torch.Tensor]:
    def convert(x: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
        return x if isinstance(x, torch.Tensor) else torch.from_numpy(x)

    return list(map(convert, args))


def adjust_dimensionality(R: npt.NDArray, Rp: npt.NDArray, strategy="zero_pad") -> Tuple[npt.NDArray, npt.NDArray]:
    D = R.shape[1]
    Dp = Rp.shape[1]
    if strategy == "zero_pad":
        if D - Dp == 0:
            return R, Rp
        elif D - Dp > 0:
            return R, np.concatenate((Rp, np.zeros((Rp.shape[0], D - Dp))), axis=1)
        else:
            return np.concatenate((R, np.zeros((R.shape[0], Dp - D))), axis=1), Rp
    else:
        raise NotImplementedError()


def center_columns(R: npt.NDArray) -> npt.NDArray:
    return R - R.mean(axis=0)[None, :]


def normalize_matrix_norm(R: npt.NDArray) -> npt.NDArray:
    return R / np.linalg.norm(R, ord="fro")


def normalize_row_norm(R: npt.NDArray) -> npt.NDArray:
    return R / np.linalg.norm(R, ord=2, axis=1, keepdims=True)


def standardize(R: npt.NDArray) -> npt.NDArray:
    return (R - R.mean(axis=0, keepdims=True)) / R.std(axis=0)


def double_center(x: npt.NDArray) -> npt.NDArray:
    return x - x.mean(axis=0, keepdims=True) - x.mean(axis=1, keepdims=True) + x.mean()


def align_spatial_dimensions(R: npt.NDArray, Rp: npt.NDArray) -> Tuple[npt.NDArray, npt.NDArray]:
    """
    Aligns spatial representations by resizing them to the smallest spatial dimension.
    Subsequent aligned spatial representations are flattened, with the spatial aligned representations
    moving into the *sample* dimension.
    """
    R_re, Rp_re = resize_wh_reps(R, Rp)
    R_re = rearrange(R_re, "n c h w -> (n h w) c")
    Rp_re = rearrange(Rp_re, "n c h w -> (n h w) c")
    if R_re.shape[0] > 5000:
        logger.info(f"Got {R_re.shape[0]} samples in N after flattening. Subsampling to reduce compute.")
        subsample = R_re.shape[0] // 5000
        R_re = R_re[::subsample]
        Rp_re = Rp_re[::subsample]

    return R_re, Rp_re


def average_pool_downsample(R, resize: bool, new_size: tuple[int, int]):
    if not resize:
        return R  # do nothing
    else:
        is_numpy = isinstance(R, np.ndarray)
        R_torch = torch.from_numpy(R) if is_numpy else R
        R_torch = torch.nn.functional.adaptive_avg_pool2d(R_torch, new_size)
        return R_torch.numpy() if is_numpy else R_torch


def resize_wh_reps(R: npt.NDArray, Rp: npt.NDArray) -> Tuple[npt.NDArray, npt.NDArray]:
    """
    Function for resizing spatial representations that are not the same size.
    Does through fourier transform and resizing.

    Args:
        R: numpy array of shape  [batch_size, height, width, num_channels]
        RP: numpy array of shape [batch_size, height, width, num_channels]

    Returns:
        fft_acts1: numpy array of shape [batch_size, (new) height, (new) width, num_channels]
        fft_acts2: numpy array of shape [batch_size, (new) height, (new) width, num_channels]

    """
    height1, width1 = R.shape[2], R.shape[3]
    height2, width2 = Rp.shape[2], Rp.shape[3]
    if height1 != height2 or width1 != width2:
        height = min(height1, height2)
        width = min(width1, width2)
        new_size = [height, width]
        resize = True
    else:
        height = height1
        width = width1
        new_size = None
        resize = False

    # resize and preprocess with fft
    avg_ds1 = average_pool_downsample(R, resize=resize, new_size=new_size)
    avg_ds2 = average_pool_downsample(Rp, resize=resize, new_size=new_size)
    return avg_ds1, avg_ds2


def fft_resize(images, resize=False, new_size=None):
    """Function for applying DFT and resizing.

    This function takes in an array of images, applies the 2-d fourier transform
    and resizes them according to new_size, keeping the frequencies that overlap
    between the two sizes.

    Args:
              images: a numpy array with shape
                      [batch_size, height, width, num_channels]
              resize: boolean, whether or not to resize
              new_size: a tuple (size, size), with height and width the same

    Returns:
              im_fft_downsampled: a numpy array with shape
                           [batch_size, (new) height, (new) width, num_channels]
    """
    assert len(images.shape) == 4, "expecting images to be" "[batch_size, height, width, num_channels]"
    if resize:
        # FFT --> remove high frequencies --> inverse FFT
        im_complex = images.astype("complex64")
        im_fft = np.fft.fft2(im_complex, axes=(1, 2))
        im_shifted = np.fft.fftshift(im_fft, axes=(1, 2))

        center_width = im_shifted.shape[2] // 2
        center_height = im_shifted.shape[1] // 2
        half_w = new_size[0] // 2
        half_h = new_size[1] // 2
        cropped_fft = im_shifted[
            :, center_height - half_h : center_height + half_h, center_width - half_w : center_width + half_w, :
        ]
        cropped_fft_shifted_back = np.fft.ifft2(cropped_fft, axes=(1, 2))
        return cropped_fft_shifted_back.real
    else:
        return images


class Pipeline:
    def __init__(
        self,
        preprocess_funcs: List[Callable[[npt.NDArray], npt.NDArray]],
        similarity_func: Callable[[npt.NDArray, npt.NDArray, SHAPE_TYPE], float],
    ) -> None:
        self.preprocess_funcs = preprocess_funcs
        self.similarity_func = similarity_func

    def __call__(self, R: npt.NDArray, Rp: npt.NDArray, shape: SHAPE_TYPE) -> float:
        try:
            for preprocess_func in self.preprocess_funcs:
                R = preprocess_func(R)
                Rp = preprocess_func(Rp)
            return self.similarity_func(R, Rp, shape)
        except ValueError as e:
            log.info(f"Pipeline failed: {e}")
            return np.nan

    def __str__(self) -> str:
        def func_name(func: Callable) -> str:
            return func.__name__ if not isinstance(func, functools.partial) else func.func.__name__

        def partial_keywords(func: Callable) -> str:
            if not isinstance(func, functools.partial):
                return ""
            else:
                return str(func.keywords)

        return (
            "Pipeline("
            + (
                "+".join(map(func_name, self.preprocess_funcs))
                + "+"
                + func_name(self.similarity_func)
                + partial_keywords(self.similarity_func)
            )
            + ")"
        )


def flatten(*args: Union[torch.Tensor, npt.NDArray], shape: SHAPE_TYPE) -> List[Union[torch.Tensor, npt.NDArray]]:
    if shape == "ntd":
        return list(map(flatten_nxtxd_to_ntxd, args))
    elif shape == "nd":
        return list(args)
    elif shape == "nchw":
        return list(map(flatten_nxcxhxw_to_nxchw, args))  # Flattening non-trivial for nchw
    else:
        raise ValueError("Unknown shape of representations. Must be one of 'ntd', 'nchw', 'nd'.")


def flatten_nxtxd_to_ntxd(R: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
    R = to_torch_if_needed(R)[0]
    log.debug("Shape before flattening: %s", str(R.shape))
    R = torch.flatten(R, start_dim=0, end_dim=1)
    log.debug("Shape after flattening: %s", str(R.shape))
    return R


def flatten_nxcxhxw_to_nxchw(R: Union[torch.Tensor, npt.NDArray]) -> torch.Tensor:
    R = to_torch_if_needed(R)[0]
    log.debug("Shape before flattening: %s", str(R.shape))
    R = torch.reshape(R, (R.shape[0], -1))
    log.debug("Shape after flattening: %s", str(R.shape))
    return R

In [ ]:
from typing import Optional
from typing import Union

import numpy as np
import numpy.typing as npt
import scipy.spatial.distance
import scipy.stats
import sklearn.metrics
import torch
# from repsim.measures.utils import flatten
# from repsim.measures.utils import RSMSimilarityMeasure
# from repsim.measures.utils import SHAPE_TYPE
# from repsim.measures.utils import to_numpy_if_needed


def representational_similarity_analysis(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    shape: SHAPE_TYPE,
    inner="correlation",
    outer="spearman",
    n_jobs: Optional[int] = None,
) -> float:
    """Representational similarity analysis

    Args:
        R (Union[torch.Tensor, npt.NDArray]): N x D representation
        Rp (Union[torch.Tensor, npt.NDArray]): N x D' representation
        inner (str, optional): inner similarity function for RSM. Must be one of
            scipy.spatial.distance.pdist identifiers . Defaults to "correlation".
        outer (str, optional): outer similarity function that compares RSMs. Defaults to
             "spearman". Must be one of "spearman", "euclidean"

    Returns:
        float: _description_
    """
    R, Rp = flatten(R, Rp, shape=shape)
    R, Rp = to_numpy_if_needed(R, Rp)

    if inner == "correlation":
        # n_jobs only works if metric is in PAIRWISE_DISTANCES as defined in sklearn, i.e., not for correlation.
        # But correlation = 1 - cosine dist of row-centered data, so we use the faster cosine metric and center the data.
        R = R - R.mean(axis=1, keepdims=True)
        S = scipy.spatial.distance.squareform(  # take the lower triangle of RSM
            1 - sklearn.metrics.pairwise_distances(R, metric="cosine", n_jobs=n_jobs),  # type:ignore
            checks=False,
        )
        Rp = Rp - Rp.mean(axis=1, keepdims=True)
        Sp = scipy.spatial.distance.squareform(
            1 - sklearn.metrics.pairwise_distances(Rp, metric="cosine", n_jobs=n_jobs),  # type:ignore
            checks=False,
        )
    elif inner == "euclidean":
        # take the lower triangle of RSM
        S = scipy.spatial.distance.squareform(
            sklearn.metrics.pairwise_distances(R, metric=inner, n_jobs=n_jobs), checks=False
        )
        Sp = scipy.spatial.distance.squareform(
            sklearn.metrics.pairwise_distances(Rp, metric=inner, n_jobs=n_jobs), checks=False
        )
    else:
        raise NotImplementedError(f"{inner=}")

    if outer == "spearman":
        return scipy.stats.spearmanr(S, Sp).statistic  # type:ignore
    elif outer == "euclidean":
        return float(np.linalg.norm(S - Sp, ord=2))
    else:
        raise ValueError(f"Unknown outer similarity function: {outer}")


class RSA(RSMSimilarityMeasure):
    def __init__(self):
        # choice of inner/outer in __call__ if fixed to default values, so these values are always the same
        super().__init__(
            sim_func=representational_similarity_analysis,
            larger_is_more_similar=True,
            is_metric=False,
            is_symmetric=True,
            invariant_to_affine=False,
            invariant_to_invertible_linear=False,
            invariant_to_ortho=False,
            invariant_to_permutation=True,
            invariant_to_isotropic_scaling=True,
            invariant_to_translation=True,
        )

In [ ]:
##################################################################################
# Copied from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/cca_core.py
# Copyright 2018 Google Inc.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""
The core code for applying Canonical Correlation Analysis to deep networks.

This module contains the core functions to apply canonical correlation analysis
to deep neural networks. The main function is get_cca_similarity, which takes in
two sets of activations, typically the neurons in two layers and their outputs
on all of the datapoints D = [d_1,...,d_m] that have been passed through.

Inputs have shape (num_neurons1, m), (num_neurons2, m). This can be directly
applied used on fully connected networks. For convolutional layers, the 3d block
of neurons can either be flattened entirely, along channels, or alternatively,
the dft_ccas (Discrete Fourier Transform) module can be used.

See:
https://arxiv.org/abs/1706.05806
https://arxiv.org/abs/1806.05759
for full details.

"""
import numpy as np
# from repsim.measures.utils import align_spatial_dimensions

num_cca_trials = 5


def positivedef_matrix_sqrt(array):
    """Stable method for computing matrix square roots, supports complex matrices.

    Args:
              array: A numpy 2d array, can be complex valued that is a positive
                     definite symmetric (or hermitian) matrix

    Returns:
              sqrtarray: The matrix square root of array
    """
    w, v = np.linalg.eigh(array)
    #  A - np.dot(v, np.dot(np.diag(w), v.T))
    wsqrt = np.sqrt(w)
    sqrtarray = np.dot(v, np.dot(np.diag(wsqrt), np.conj(v).T))
    return sqrtarray


def remove_small(sigma_xx, sigma_xy, sigma_yx, sigma_yy, epsilon):
    """Takes covariance between X, Y, and removes values of small magnitude.

    Args:
              sigma_xx: 2d numpy array, variance matrix for x
              sigma_xy: 2d numpy array, crossvariance matrix for x,y
              sigma_yx: 2d numpy array, crossvariance matrixy for x,y,
                        (conjugate) transpose of sigma_xy
              sigma_yy: 2d numpy array, variance matrix for y
              epsilon : cutoff value for norm below which directions are thrown
                         away

    Returns:
              sigma_xx_crop: 2d array with low x norm directions removed
              sigma_xy_crop: 2d array with low x and y norm directions removed
              sigma_yx_crop: 2d array with low x and y norm directiosn removed
              sigma_yy_crop: 2d array with low y norm directions removed
              x_idxs: indexes of sigma_xx that were removed
              y_idxs: indexes of sigma_yy that were removed
    """

    x_diag = np.abs(np.diagonal(sigma_xx))
    y_diag = np.abs(np.diagonal(sigma_yy))
    x_idxs = x_diag >= epsilon
    y_idxs = y_diag >= epsilon

    sigma_xx_crop = sigma_xx[x_idxs][:, x_idxs]
    sigma_xy_crop = sigma_xy[x_idxs][:, y_idxs]
    sigma_yx_crop = sigma_yx[y_idxs][:, x_idxs]
    sigma_yy_crop = sigma_yy[y_idxs][:, y_idxs]

    return (sigma_xx_crop, sigma_xy_crop, sigma_yx_crop, sigma_yy_crop, x_idxs, y_idxs)


def compute_ccas(sigma_xx, sigma_xy, sigma_yx, sigma_yy, epsilon, verbose=True):
    """Main cca computation function, takes in variances and crossvariances.

    This function takes in the covariances and cross covariances of X, Y,
    preprocesses them (removing small magnitudes) and outputs the raw results of
    the cca computation, including cca directions in a rotated space, and the
    cca correlation coefficient values.

    Args:
              sigma_xx: 2d numpy array, (num_neurons_x, num_neurons_x)
                        variance matrix for x
              sigma_xy: 2d numpy array, (num_neurons_x, num_neurons_y)
                        crossvariance matrix for x,y
              sigma_yx: 2d numpy array, (num_neurons_y, num_neurons_x)
                        crossvariance matrix for x,y (conj) transpose of sigma_xy
              sigma_yy: 2d numpy array, (num_neurons_y, num_neurons_y)
                        variance matrix for y
              epsilon:  small float to help with stabilizing computations
              verbose:  boolean on whether to print intermediate outputs

    Returns:
              [ux, sx, vx]: [numpy 2d array, numpy 1d array, numpy 2d array]
                            ux and vx are (conj) transposes of each other, being
                            the canonical directions in the X subspace.
                            sx is the set of canonical correlation coefficients-
                            how well corresponding directions in vx, Vy correlate
                            with each other.
              [uy, sy, vy]: Same as above, but for Y space
              invsqrt_xx:   Inverse square root of sigma_xx to transform canonical
                            directions back to original space
              invsqrt_yy:   Same as above but for sigma_yy
              x_idxs:       The indexes of the input sigma_xx that were pruned
                            by remove_small
              y_idxs:       Same as above but for sigma_yy
    """

    (sigma_xx, sigma_xy, sigma_yx, sigma_yy, x_idxs, y_idxs) = remove_small(
        sigma_xx, sigma_xy, sigma_yx, sigma_yy, epsilon
    )

    numx = sigma_xx.shape[0]
    numy = sigma_yy.shape[0]

    if numx == 0 or numy == 0:
        return (
            [0, 0, 0],
            [0, 0, 0],
            np.zeros_like(sigma_xx),
            np.zeros_like(sigma_yy),
            x_idxs,
            y_idxs,
        )

    if verbose:
        print("adding eps to diagonal and taking inverse")
    sigma_xx += epsilon * np.eye(numx)
    sigma_yy += epsilon * np.eye(numy)
    inv_xx = np.linalg.pinv(sigma_xx)
    inv_yy = np.linalg.pinv(sigma_yy)

    if verbose:
        print("taking square root")
    invsqrt_xx = positivedef_matrix_sqrt(inv_xx)
    invsqrt_yy = positivedef_matrix_sqrt(inv_yy)

    if verbose:
        print("dot products...")
    arr = np.dot(invsqrt_xx, np.dot(sigma_xy, invsqrt_yy))

    if verbose:
        print("trying to take final svd")
    u, s, v = np.linalg.svd(arr)

    if verbose:
        print("computed everything!")

    return [u, np.abs(s), v], invsqrt_xx, invsqrt_yy, x_idxs, y_idxs


def sum_threshold(array, threshold):
    """Computes threshold index of decreasing nonnegative array by summing.

    This function takes in a decreasing array nonnegative floats, and a
    threshold between 0 and 1. It returns the index i at which the sum of the
    array up to i is threshold*total mass of the array.

    Args:
              array: a 1d numpy array of decreasing, nonnegative floats
              threshold: a number between 0 and 1

    Returns:
              i: index at which np.sum(array[:i]) >= threshold
    """
    assert (threshold >= 0) and (threshold <= 1), "print incorrect threshold"

    for i in range(len(array)):
        if np.sum(array[:i]) / np.sum(array) >= threshold:
            return i


def create_zero_dict(compute_dirns, dimension):
    """Outputs a zero dict when neuron activation norms too small.

    This function creates a return_dict with appropriately shaped zero entries
    when all neuron activations are very small.

    Args:
              compute_dirns: boolean, whether to have zero vectors for directions
              dimension: int, defines shape of directions

    Returns:
              return_dict: a dict of appropriately shaped zero entries
    """
    return_dict = {}
    return_dict["mean"] = (np.asarray(0), np.asarray(0))
    return_dict["sum"] = (np.asarray(0), np.asarray(0))
    return_dict["cca_coef1"] = np.asarray(0)
    return_dict["cca_coef2"] = np.asarray(0)
    return_dict["idx1"] = 0
    return_dict["idx2"] = 0

    if compute_dirns:
        return_dict["cca_dirns1"] = np.zeros((1, dimension))
        return_dict["cca_dirns2"] = np.zeros((1, dimension))

    return return_dict


def get_cca_similarity(
    acts1,
    acts2,
    epsilon=0.0,
    threshold=0.98,
    compute_coefs=True,
    compute_dirns=False,
    verbose=True,
):
    """The main function for computing cca similarities.

    This function computes the cca similarity between two sets of activations,
    returning a dict with the cca coefficients, a few statistics of the cca
    coefficients, and (optionally) the actual directions.

    Args:
              acts1: (num_neurons1, data_points) a 2d numpy array of neurons by
                     datapoints where entry (i,j) is the output of neuron i on
                     datapoint j.
              acts2: (num_neurons2, data_points) same as above, but (potentially)
                     for a different set of neurons. Note that acts1 and acts2
                     can have different numbers of neurons, but must agree on the
                     number of datapoints

              epsilon: small float to help stabilize computations

              threshold: float between 0, 1 used to get rid of trailing zeros in
                         the cca correlation coefficients to output more accurate
                         summary statistics of correlations.


              compute_coefs: boolean value determining whether coefficients
                             over neurons are computed. Needed for computing
                             directions

              compute_dirns: boolean value determining whether actual cca
                             directions are computed. (For very large neurons and
                             datasets, may be better to compute these on the fly
                             instead of store in memory.)

              verbose: Boolean, whether intermediate outputs are printed

    Returns:
              return_dict: A dictionary with outputs from the cca computations.
                           Contains neuron coefficients (combinations of neurons
                           that correspond to cca directions), the cca correlation
                           coefficients (how well aligned directions correlate),
                           x and y idxs (for computing cca directions on the fly
                           if compute_dirns=False), and summary statistics. If
                           compute_dirns=True, the cca directions are also
                           computed.
    """

    # assert dimensionality equal
    assert acts1.shape[1] == acts2.shape[1], "dimensions don't match"
    # check that acts1, acts2 are transposition
    assert acts1.shape[0] < acts1.shape[1], "input must be number of neurons" "by datapoints"
    return_dict = {}

    # compute covariance with numpy function for extra stability
    numx = acts1.shape[0]
    numy = acts2.shape[0]

    covariance = np.cov(acts1, acts2)
    sigmaxx = covariance[:numx, :numx]
    sigmaxy = covariance[:numx, numx:]
    sigmayx = covariance[numx:, :numx]
    sigmayy = covariance[numx:, numx:]

    # rescale covariance to make cca computation more stable
    xmax = np.max(np.abs(sigmaxx))
    ymax = np.max(np.abs(sigmayy))
    sigmaxx /= xmax
    sigmayy /= ymax
    sigmaxy /= np.sqrt(xmax * ymax)
    sigmayx /= np.sqrt(xmax * ymax)

    ([u, s, v], invsqrt_xx, invsqrt_yy, x_idxs, y_idxs) = compute_ccas(
        sigmaxx, sigmaxy, sigmayx, sigmayy, epsilon=epsilon, verbose=verbose
    )

    # if x_idxs or y_idxs is all false, return_dict has zero entries
    if (not np.any(x_idxs)) or (not np.any(y_idxs)):
        return create_zero_dict(compute_dirns, acts1.shape[1])

    if compute_coefs:
        # also compute full coefficients over all neurons
        x_mask = np.dot(x_idxs.reshape((-1, 1)), x_idxs.reshape((1, -1)))
        y_mask = np.dot(y_idxs.reshape((-1, 1)), y_idxs.reshape((1, -1)))

        return_dict["coef_x"] = u.T
        return_dict["invsqrt_xx"] = invsqrt_xx
        return_dict["full_coef_x"] = np.zeros((numx, numx))
        np.place(return_dict["full_coef_x"], x_mask, return_dict["coef_x"])
        return_dict["full_invsqrt_xx"] = np.zeros((numx, numx))
        np.place(return_dict["full_invsqrt_xx"], x_mask, return_dict["invsqrt_xx"])

        return_dict["coef_y"] = v
        return_dict["invsqrt_yy"] = invsqrt_yy
        return_dict["full_coef_y"] = np.zeros((numy, numy))
        np.place(return_dict["full_coef_y"], y_mask, return_dict["coef_y"])
        return_dict["full_invsqrt_yy"] = np.zeros((numy, numy))
        np.place(return_dict["full_invsqrt_yy"], y_mask, return_dict["invsqrt_yy"])

        # compute means
        neuron_means1 = np.mean(acts1, axis=1, keepdims=True)
        neuron_means2 = np.mean(acts2, axis=1, keepdims=True)
        return_dict["neuron_means1"] = neuron_means1
        return_dict["neuron_means2"] = neuron_means2

    if compute_dirns:
        # orthonormal directions that are CCA directions
        cca_dirns1 = (
            np.dot(
                np.dot(return_dict["full_coef_x"], return_dict["full_invsqrt_xx"]),
                (acts1 - neuron_means1),
            )
            + neuron_means1
        )
        cca_dirns2 = (
            np.dot(
                np.dot(return_dict["full_coef_y"], return_dict["full_invsqrt_yy"]),
                (acts2 - neuron_means2),
            )
            + neuron_means2
        )

    # get rid of trailing zeros in the cca coefficients
    idx1 = sum_threshold(s, threshold)
    idx2 = sum_threshold(s, threshold)

    return_dict["cca_coef1"] = s
    return_dict["cca_coef2"] = s
    return_dict["x_idxs"] = x_idxs
    return_dict["y_idxs"] = y_idxs
    # summary statistics
    return_dict["mean"] = (np.mean(s[:idx1]), np.mean(s[:idx2]))
    return_dict["sum"] = (np.sum(s), np.sum(s))

    if compute_dirns:
        return_dict["cca_dirns1"] = cca_dirns1
        return_dict["cca_dirns2"] = cca_dirns2

    return return_dict


def robust_cca_similarity(acts1, acts2, threshold=0.98, epsilon=1e-6, compute_dirns=True):
    """Calls get_cca_similarity multiple times while adding noise.

    This function is very similar to get_cca_similarity, and can be used if
    get_cca_similarity doesn't converge for some pair of inputs. This function
    adds some noise to the activations to help convergence.

    Args:
              acts1: (num_neurons1, data_points) a 2d numpy array of neurons by
                     datapoints where entry (i,j) is the output of neuron i on
                     datapoint j.
              acts2: (num_neurons2, data_points) same as above, but (potentially)
                     for a different set of neurons. Note that acts1 and acts2
                     can have different numbers of neurons, but must agree on the
                     number of datapoints

              threshold: float between 0, 1 used to get rid of trailing zeros in
                         the cca correlation coefficients to output more accurate
                         summary statistics of correlations.

              epsilon: small float to help stabilize computations

              compute_dirns: boolean value determining whether actual cca
                             directions are computed. (For very large neurons and
                             datasets, may be better to compute these on the fly
                             instead of store in memory.)

    Returns:
              return_dict: A dictionary with outputs from the cca computations.
                           Contains neuron coefficients (combinations of neurons
                           that correspond to cca directions), the cca correlation
                           coefficients (how well aligned directions correlate),
                           x and y idxs (for computing cca directions on the fly
                           if compute_dirns=False), and summary statistics. If
                           compute_dirns=True, the cca directions are also
                           computed.
    """

    for trial in range(num_cca_trials):
        try:
            return_dict = get_cca_similarity(acts1, acts2, threshold, compute_dirns)
        except np.linalg.LinAlgError:
            acts1 = acts1 * 1e-1 + np.random.normal(size=acts1.shape) * epsilon
            acts2 = acts2 * 1e-1 + np.random.normal(size=acts1.shape) * epsilon
            if trial + 1 == num_cca_trials:
                raise

    return return_dict
    # End of copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/cca_core.py


def top_k_pca_comps(singular_values, threshold=0.99):
    total_variance = np.sum(singular_values**2)
    explained_variance = (singular_values**2) / total_variance
    cumulative_variance = np.cumsum(explained_variance)
    return np.argmax(cumulative_variance >= threshold * total_variance) + 1


def _svcca_original(acts1, acts2):
    # Copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/tutorials/001_Introduction.ipynb
    # Modification: get_cca_similarity is in the same file.
    # Modification: top-k PCA component selection s.t. explained variance > 0.99 total variance
    # Mean subtract activations
    cacts1 = acts1 - np.mean(acts1, axis=1, keepdims=True)
    cacts2 = acts2 - np.mean(acts2, axis=1, keepdims=True)

    # Perform SVD
    U1, s1, V1 = np.linalg.svd(cacts1, full_matrices=False)
    U2, s2, V2 = np.linalg.svd(cacts2, full_matrices=False)

    # top-k PCA components only
    k1 = top_k_pca_comps(s1)
    k2 = top_k_pca_comps(s2)

    svacts1 = np.dot(s1[:k1] * np.eye(k1), V1[:k1])
    # can also compute as svacts1 = np.dot(U1.T[:20], cacts1)
    svacts2 = np.dot(s2[:k2] * np.eye(k2), V2[:k2])
    # can also compute as svacts1 = np.dot(U2.T[:20], cacts2)

    svcca_results = get_cca_similarity(svacts1, svacts2, epsilon=1e-10, verbose=False)
    # End of copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/tutorials/001_Introduction.ipynb
    return np.mean(svcca_results["cca_coef1"])


# Copied from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/pwcca.py
# Modification: get_cca_similarity is in the same file.
def compute_pwcca(acts1, acts2, epsilon=0.0):
    """Computes projection weighting for weighting CCA coefficients

    Args:
         acts1: 2d numpy array, shaped (neurons, num_datapoints)
         acts2: 2d numpy array, shaped (neurons, num_datapoints)

    Returns:
         Original cca coefficient mean and weighted mean

    """
    sresults = get_cca_similarity(
        acts1,
        acts2,
        epsilon=epsilon,
        compute_dirns=False,
        compute_coefs=True,
        verbose=False,
    )
    if np.sum(sresults["x_idxs"]) <= np.sum(sresults["y_idxs"]):
        dirns = (
            np.dot(
                sresults["coef_x"],
                (acts1[sresults["x_idxs"]] - sresults["neuron_means1"][sresults["x_idxs"]]),
            )
            + sresults["neuron_means1"][sresults["x_idxs"]]
        )
        coefs = sresults["cca_coef1"]
        acts = acts1
        idxs = sresults["x_idxs"]
    else:
        dirns = (
            np.dot(
                sresults["coef_y"],
                (acts1[sresults["y_idxs"]] - sresults["neuron_means2"][sresults["y_idxs"]]),
            )
            + sresults["neuron_means2"][sresults["y_idxs"]]
        )
        coefs = sresults["cca_coef2"]
        acts = acts2
        idxs = sresults["y_idxs"]
    P, _ = np.linalg.qr(dirns.T)
    weights = np.sum(np.abs(np.dot(P.T, acts[idxs].T)), axis=1)
    weights = weights / np.sum(weights)

    return np.sum(weights * coefs), weights, coefs
    # End of copy from https://github.com/google/svcca/blob/1f3fbf19bd31bd9b76e728ef75842aa1d9a4cd2b/pwcca.py


##################################################################################

from typing import Union  # noqa:e402

import numpy.typing as npt  # noqa:e402
import torch  # noqa:e402

# from repsim.measures.utils import (
#     SHAPE_TYPE,
#     flatten,
#     resize_wh_reps,
#     to_numpy_if_needed,
#     RepresentationalSimilarityMeasure,
# )  # noqa:e402


def svcca(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    shape: SHAPE_TYPE,
) -> float:
    R, Rp = flatten(R, Rp, shape=shape)
    R, Rp = to_numpy_if_needed(R, Rp)
    return _svcca_original(R.T, Rp.T)


def pwcca(
    R: Union[torch.Tensor, npt.NDArray],
    Rp: Union[torch.Tensor, npt.NDArray],
    shape: SHAPE_TYPE,
) -> float:
    R, Rp = flatten(R, Rp, shape=shape)
    R, Rp = to_numpy_if_needed(R, Rp)
    return compute_pwcca(R.T, Rp.T)[0]


class SVCCA(RepresentationalSimilarityMeasure):
    def __init__(self):
        super().__init__(
            sim_func=svcca,
            larger_is_more_similar=True,
            is_metric=False,
            is_symmetric=True,
            invariant_to_affine=False,
            invariant_to_invertible_linear=False,
            invariant_to_ortho=True,
            invariant_to_permutation=True,
            invariant_to_isotropic_scaling=True,
            invariant_to_translation=True,
        )

    def __call__(self, R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray, shape: SHAPE_TYPE) -> float:
        if shape == "nchw":
            # Move spatial dimensions into the sample dimension
            # If not the same spatial dimension, resample via FFT.
            R, Rp = align_spatial_dimensions(R, Rp)
            shape = "nd"

        return self.sim_func(R, Rp, shape)


class PWCCA(RepresentationalSimilarityMeasure):
    def __init__(self):
        super().__init__(
            sim_func=pwcca,
            larger_is_more_similar=True,
            is_metric=False,
            is_symmetric=False,
            invariant_to_affine=False,
            invariant_to_invertible_linear=False,
            invariant_to_ortho=False,
            invariant_to_permutation=False,
            invariant_to_isotropic_scaling=True,
            invariant_to_translation=True,
        )

    def __call__(self, R: torch.Tensor | npt.NDArray, Rp: torch.Tensor | npt.NDArray, shape: SHAPE_TYPE) -> float:
        if shape == "nchw":
            # Move spatial dimensions into the sample dimension
            # If not the same spatial dimension, resample via FFT.
            R, Rp = align_spatial_dimensions(R, Rp)
            shape = "nd"

        return self.sim_func(R, Rp, shape)

## get rand

In [ ]:
def score_rand(num_feats, sim_fn, shapereq_bool=False):
    all_rand_scores = []
    # num_feats = len(uniq_corr_indices_AB_forA)
    for i in range(10):
        rand_modA_feats = np.random.randint(low=0, high=weight_matrix.shape[0], size=num_feats).tolist()
        rand_modB_feats = np.random.randint(low=0, high=weight_matrix.shape[0], size=num_feats).tolist()

        if shapereq_bool:
            score = sim_fn(weight_matrix[rand_modA_feats], weight_matrix[rand_modB_feats], "nd")
        else:
            score = sim_fn(weight_matrix[rand_modA_feats], weight_matrix[rand_modB_feats])
        all_rand_scores.append(score)
    print(sum(all_rand_scores) / len(all_rand_scores))
    # plt.hist(all_rand_scores)
    # plt.show()
    return sum(all_rand_scores) / len(all_rand_scores)

# load labels

In [48]:
import json
with open('gemma-2-2b-20-gemmascope-res-16k-explanations.json', 'rb') as f:
    feat_labels_allData = json.load(f)

In [49]:
feat_labels_lst = [0 for i in range(feat_labels_allData['explanationsCount'])]
feat_labels_dict = {}
for f_dict in feat_labels_allData['explanations']:
    feat_labels_lst[int(f_dict['index'])] = f_dict['description']
    feat_labels_dict[int(f_dict['index'])] = f_dict['description']
    if int(f_dict['index']) == 0:
        print(f_dict['description'])

mentions of the letter 'Z' and variations of 'Z' in different contexts


In [50]:
len(feat_labels_dict)

16383

# search for features

In [ ]:
def find_indices_with_keyword(f_dict, keyword):
    """
    Find all indices of fList which contain the keyword in the string at those indices.

    Args:
    fList (list of str): List of strings to search within.
    keyword (str): Keyword to search for within the strings of fList.

    Returns:
    list of int: List of indices where the keyword is found within the strings of fList.
    """
    filt_dict = {}
    for index, string in f_dict.items():
        # split_list = string.split(',')
        # no_space_list = [i.replace(' ', '').lower() for i in split_list]
        # if keyword in no_space_list:
        if keyword in string:
            filt_dict[index] = string
    return filt_dict

In [ ]:
keyword = "number"
number_feats = find_indices_with_keyword(feat_labels_dict, keyword)

In [ ]:
keyword = "month"
month_feats = find_indices_with_keyword(feat_labels_dict, keyword)

In [ ]:
set(number_feats).intersection(month_feats)

{5769, 12790, 20260, 21029}

In [ ]:
number_feats[5769]

'numbers with possible time expressions like years, months, and hours'

In [ ]:
for common_feat in set(number_feats).intersection(month_feats):
    del number_feats[common_feat]

In [ ]:
set(number_feats).intersection(month_feats)

set()

# load model

In [4]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig, AutoTokenizer
import numpy as np
import torch

In [5]:
from huggingface_hub import hf_hub_download, notebook_login
notebook_login()

In [6]:
torch.set_grad_enabled(False) # avoid blowing up mem

In [7]:
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b",
    device_map='auto',
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [8]:
tokenizer =  AutoTokenizer.from_pretrained("google/gemma-2-2b")

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

# load sae

In [22]:
path_to_params = hf_hub_download(
    repo_id="google/gemma-scope-2b-pt-res",
    filename="layer_20/width_16k/average_l0_71/params.npz",
    force_download=False,
)

params.npz:   0%|          | 0.00/302M [00:00<?, ?B/s]

In [23]:
params = np.load(path_to_params)
pt_params = {k: torch.from_numpy(v).cuda() for k, v in params.items()}

In [24]:
import torch.nn as nn
class JumpReLUSAE(nn.Module):
  def __init__(self, d_model, d_sae):
    # Note that we initialise these to zeros because we're loading in pre-trained weights.
    # If you want to train your own SAEs then we recommend using blah
    super().__init__()
    self.W_enc = nn.Parameter(torch.zeros(d_model, d_sae))
    self.W_dec = nn.Parameter(torch.zeros(d_sae, d_model))
    self.threshold = nn.Parameter(torch.zeros(d_sae))
    self.b_enc = nn.Parameter(torch.zeros(d_sae))
    self.b_dec = nn.Parameter(torch.zeros(d_model))

  def encode(self, input_acts):
    pre_acts = input_acts @ self.W_enc + self.b_enc
    mask = (pre_acts > self.threshold)
    acts = mask * torch.nn.functional.relu(pre_acts)
    return acts

  def decode(self, acts):
    return acts @ self.W_dec + self.b_dec

  def forward(self, acts):
    acts = self.encode(acts)
    recon = self.decode(acts)
    return recon


In [25]:
sae = JumpReLUSAE(params['W_enc'].shape[0], params['W_enc'].shape[1])
sae.load_state_dict(pt_params)

<All keys matched successfully>

In [36]:
sae.cuda()

JumpReLUSAE()

In [ ]:
# layer_name = "blocks.17.hook_resid_post"

# sae, cfg_dict, sparsity = SAE.from_pretrained(
#     release = "google/gemma-scope-2b-pt-res",
#     sae_id = layer_name,
#     device = device
# )

# test prompts

In [ ]:
prompt = "one two three four"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=1)
print(tokenizer.decode(outputs[0, -1]))

 five


In [ ]:
prompt = "January February March April"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=1)
print(tokenizer.decode(outputs[0, -1]))

 May


In [ ]:
prompt = "enero febrero marzo abril"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=1)
print(tokenizer.decode(outputs[0, -1]))

 mayo


In [110]:
prompt = "My favorite animal is a"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=1)
print(tokenizer.decode(outputs[0, -1]))

 dog


In [ ]:
prompt = "My least favorite animal is a"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=1)
print(tokenizer.decode(outputs[0, -1]))

 snake


In [128]:
prompt = "I like dogs. What animal do I like? A: dog. I like cats. What animal do I like? A: "
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=1)
print(tokenizer.decode(outputs[0, -1]))

<strong>


In [129]:
prompt = "I love"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=1)
print(tokenizer.decode(outputs[0, -1]))

 this


In [130]:
prompt = "I hate"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=1)
print(tokenizer.decode(outputs[0, -1]))

 to


In [162]:
prompt = "The capital of Washington is Seattle. The capital of France is"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=1)
print(tokenizer.decode(outputs[0, -1]))

 Paris


In [163]:
prompt = "The capital of Washington is Seattle. The capital of Germany is"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=1)
print(tokenizer.decode(outputs[0, -1]))

 Berlin


In [182]:
prompt = "The capital of France is Paris. The capital of China is"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=1)
print(tokenizer.decode(outputs[0, -1]))

 Beijing


In [183]:
prompt = "The capital of France is Paris. The capital of America is"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
outputs = model.generate(input_ids=inputs, max_new_tokens=1)
print(tokenizer.decode(outputs[0, -1]))

 Washington


# get actv fns

In [27]:
prompt = "January February March April"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
# outputs = model.generate(input_ids=inputs, max_new_tokens=1)
# print(tokenizer.decode(outputs[0, -1]))

In [28]:
def gather_residual_activations(model, target_layer, inputs):
  target_act = None
  def gather_target_act_hook(mod, inputs, outputs):
    nonlocal target_act # make sure we can modify the target_act from the outer scope
    target_act = outputs[0]
    return outputs
  handle = model.model.layers[target_layer].register_forward_hook(gather_target_act_hook)
  _ = model.forward(inputs)
  handle.remove()
  return target_act

In [29]:
target_act = gather_residual_activations(model, 20, inputs)

Now, we can run our SAE on the saved activations.

In [30]:
sae_acts = sae.encode(target_act.to(torch.float32))
# recon = sae.decode(sae_acts)

In [ ]:
sae_acts.shape

torch.Size([1, 5, 16384])

# load sae weights


In [31]:
weight_matrix = sae.W_dec.detach().cpu().numpy()
weight_matrix.shape

(16384, 2304)

# find common feats (single samp)

## fav vs least fav animal

In [111]:
prompt = "My favorite animal is a"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_1 = sae.encode(target_act.to(torch.float32))

In [112]:
prompt = "My least favorite animal is a"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_2 = sae.encode(target_act.to(torch.float32))

In [113]:
sae_acts_1.shape

torch.Size([1, 6, 16384])

In [114]:
feat_k = 15
one_top_acts_values, one_top_acts_indices = sae_acts_1[0, -1, :].topk(feat_k, dim=-1)
one_top_acts_indices.sort().values

tensor([  247,   259,  1089,  1692,  4835,  6631,  7071,  7977,  8859,  9768,
         9982, 12120, 12329, 15175, 15925], device='cuda:0')

In [115]:
two_top_acts_values, two_top_acts_indices = sae_acts_2[0, -1, :].topk(feat_k, dim=-1)
two_top_acts_indices.sort().values

tensor([  259,  1089,  1692,  3770,  4243,  4844,  5203,  5333,  7071,  8636,
         9768,  9982, 13996, 15175, 15925], device='cuda:0')

In [116]:
common_feats = set((one_top_acts_indices).tolist()).intersection(set((two_top_acts_indices).tolist()))
common_feats

{259, 1089, 1692, 7071, 9768, 9982, 15175, 15925}

In [117]:
for f_ind in common_feats:
    print(f_ind, feat_labels_lst[f_ind])

1089 terms related to dogs and their training or behavioral characteristics
259 mathematical comparisons and relationships, especially involving values and operations
15175 references to wildlife and their interactions in natural environments
9768 terms related to control and authority, particularly in political or systemic contexts
15925 references to personal identity or self-description
1692 legal and technical terminology related to statutes and inventions
9982 instances of the article "a" and its variations in text
7071  the presence of the letter 'a' in various contexts


### top 50

In [118]:
feat_k = 50
one_top_acts_values, one_top_acts_indices = sae_acts_1[0, -1, :].topk(feat_k, dim=-1)
# one_top_acts_indices.sort().values

In [119]:
two_top_acts_values, two_top_acts_indices = sae_acts_2[0, -1, :].topk(feat_k, dim=-1)
# two_top_acts_indices.sort().values

In [121]:
for val, ind in zip(one_top_acts_values, one_top_acts_indices):
    print(round(val.item(), 2), ind.item(), feat_labels_lst[ind])

89.22 15175 references to wildlife and their interactions in natural environments
74.91 9982 instances of the article "a" and its variations in text
45.71 1089 terms related to dogs and their training or behavioral characteristics
37.76 9768 terms related to control and authority, particularly in political or systemic contexts
31.59 1692 legal and technical terminology related to statutes and inventions
23.45 6631 the beginning of a text or important markers in a document
23.02 15925 references to personal identity or self-description
22.13 12120 references to plant species and their classifications
21.42 7977 elements related to food items and their descriptions
20.6 259 mathematical comparisons and relationships, especially involving values and operations
17.88 8859 terminology related to virus behaviors and characteristics
17.0 247 references to institutions or locations in a formal context, particularly focusing on educational and financial entities
16.22 7071  the presence of the 

In [120]:
for val, ind in zip(two_top_acts_values, two_top_acts_indices):
    print(round(val.item(), 2), ind.item(), feat_labels_lst[ind])

77.45 9982 instances of the article "a" and its variations in text
71.5 15175 references to wildlife and their interactions in natural environments
36.54 9768 terms related to control and authority, particularly in political or systemic contexts
26.35 1692 legal and technical terminology related to statutes and inventions
23.95 15925 references to personal identity or self-description
23.81 4243 phrases related to sports results, particularly draws and matches
22.11 259 mathematical comparisons and relationships, especially involving values and operations
20.04 4844 definitions and descriptions related to terms and concepts
19.33 7071  the presence of the letter 'a' in various contexts
18.42 5333 phrases describing various qualities or attributes, often with an emphasis on positivity or significance
18.38 1089 terms related to dogs and their training or behavioral characteristics
17.59 13996 species-related terminology in biological contexts
16.22 5203 specific quantities or occurrence

In [122]:
common_feats = set((one_top_acts_indices).tolist()).intersection(set((two_top_acts_indices).tolist()))
# common_feats

In [123]:
len(common_feats)

36

In [124]:
for f_ind in common_feats:
    print(f_ind, feat_labels_lst[f_ind])

259 mathematical comparisons and relationships, especially involving values and operations
13700 references to species comparisons and interactions, particularly between humans and other mammals
5509  concepts related to affirmative action and its implications
4243 phrases related to sports results, particularly draws and matches
1815 references to notable individuals and their achievements
8859 terminology related to virus behaviors and characteristics
1692 legal and technical terminology related to statutes and inventions
5533 terms related to medical and biological testing and analysis
7455 determiners and quantifiers in various contexts
7071  the presence of the letter 'a' in various contexts
9768 terms related to control and authority, particularly in political or systemic contexts
12329  statements about violation of rules or regulations
7977 elements related to food items and their descriptions
1583 words related to administrative or technical tasks and issues
3636 emotional exp

In [125]:
nums_only = set((one_top_acts_indices).tolist()) - set((two_top_acts_indices).tolist())
for f_ind in nums_only:
    print(f_ind, feat_labels_lst[f_ind])

6788  mentions of endangered species and conservation efforts
549 text that involves programming and code structure
3518 phrases indicating disapproval or violation of rules
6631 the beginning of a text or important markers in a document
9704 sections of text related to events and outcomes
6698 references to specific dates and numerical data within the text
16139 terms and phrases related to wildlife, specifically marine and avian species
15572 instances of new or modern vehicles
6356 identifications and descriptions of individuals, particularly in a sporting or performance context
6772 mentions of cats in various contexts
3831 references to health-related issues and medical conditions
8508 terms related to medical conditions and health implications
3997 personal reflections on life experiences and relationships
5278 elements related to visual design and detail


In [126]:
months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
for f_ind in months_only:
    print(f_ind, feat_labels_lst[f_ind])

15232 references to diseases and viruses transmitted by mosquitoes or ticks
3234 specific references to criminal cases and legal terminology
16356  phrases and concepts related to specific instances or entities in various contexts
16137 expressions related to emotional and existential boredom or dissatisfaction
13324 expressions of strong emotions and expletives
13996 species-related terminology in biological contexts
15027 references to personal transformation and significant life experiences
14772 the concept of "Digital Twins" and their applications
13623 phrases that describe significant or notable subjects or events
9624 terms related to the characteristics and behavior of prehistoric reptiles, specifically crocodilians
6011  expressions related to reflective and observational experiences
8636 elements related to characters' roles and abilities in fantasy narratives
6301 references to specific quantities or degrees
15167  themes related to youth struggles and social issues


## I love vs I hate

In [131]:
prompt = "I love"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_1 = sae.encode(target_act.to(torch.float32))

In [132]:
prompt = "I hate"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_2 = sae.encode(target_act.to(torch.float32))

In [133]:
sae_acts_1.shape

torch.Size([1, 3, 16384])

### top 15

In [134]:
feat_k = 15
one_top_acts_values, one_top_acts_indices = sae_acts_1[0, -1, :].topk(feat_k, dim=-1)
one_top_acts_indices.sort().values

tensor([ 2230,  3019,  3840,  3997,  5426,  5525,  6631,  7541,  8713,  9362,
         9602,  9768, 12967, 13107, 16099], device='cuda:0')

In [135]:
two_top_acts_values, two_top_acts_indices = sae_acts_2[0, -1, :].topk(feat_k, dim=-1)
two_top_acts_indices.sort().values

tensor([  139,  2604,  3019,  3840,  3997,  5426,  6631,  7541,  7971,  9362,
         9768, 12967, 13107, 13135, 13566], device='cuda:0')

In [136]:
common_feats = set((one_top_acts_indices).tolist()).intersection(set((two_top_acts_indices).tolist()))
common_feats

{3019, 3840, 3997, 5426, 6631, 7541, 9362, 9768, 12967, 13107}

In [137]:
for f_ind in common_feats:
    print(f_ind, feat_labels_lst[f_ind])

3840 phrases related to opportunities and suggestions for improvement
12967 references to personal interests and passions
9768 terms related to control and authority, particularly in political or systemic contexts
6631 the beginning of a text or important markers in a document
3019  elements related to operational or procedural contexts in a structured format
9362  expressions of affection or appreciation
5426  references to collaboration and collective effort in various contexts
13107 expressions of strong negative sentiments or opinions
7541 emotional expressions and the perception of feelings
3997 personal reflections on life experiences and relationships


### top 50

In [138]:
feat_k = 50
one_top_acts_values, one_top_acts_indices = sae_acts_1[0, -1, :].topk(feat_k, dim=-1)
# one_top_acts_indices.sort().values

In [139]:
two_top_acts_values, two_top_acts_indices = sae_acts_2[0, -1, :].topk(feat_k, dim=-1)
# two_top_acts_indices.sort().values

In [140]:
for val, ind in zip(one_top_acts_values, one_top_acts_indices):
    print(round(val.item(), 2), ind.item(), feat_labels_lst[ind])

113.25 9362  expressions of affection or appreciation
64.33 6631 the beginning of a text or important markers in a document
60.31 12967 references to personal interests and passions
56.81 9602 words and themes related to love and affection
48.57 3997 personal reflections on life experiences and relationships
45.4 9768 terms related to control and authority, particularly in political or systemic contexts
42.24 8713 phrases that indicate addressing or overcoming challenges
40.63 3019  elements related to operational or procedural contexts in a structured format
39.03 3840 phrases related to opportunities and suggestions for improvement
37.3 5426  references to collaboration and collective effort in various contexts
25.15 13107 expressions of strong negative sentiments or opinions
24.88 7541 emotional expressions and the perception of feelings
24.52 16099 quantitative descriptors indicating abundance or quantity
23.88 5525 terms related to assessment and improvement in various contexts
20

In [141]:
for val, ind in zip(two_top_acts_values, two_top_acts_indices):
    print(round(val.item(), 2), ind.item(), feat_labels_lst[ind])

75.36 9362  expressions of affection or appreciation
67.41 2604 phrases expressing dislike or hatred towards people or situations.
48.07 13135 expressions of negativity or unfavorable situations
47.8 6631 the beginning of a text or important markers in a document
44.73 9768 terms related to control and authority, particularly in political or systemic contexts
41.01 3997 personal reflections on life experiences and relationships
36.79 139 references to actions and their consequences, emphasizing occurrences and substances
35.8 3019  elements related to operational or procedural contexts in a structured format
35.1 3840 phrases related to opportunities and suggestions for improvement
32.65 13566 terms related to "hat" and its variations, including forms of the word such as "hating."
29.33 5426  references to collaboration and collective effort in various contexts
29.25 13107 expressions of strong negative sentiments or opinions
26.96 7971  references to specific scientific articles and r

In [142]:
common_feats = set((one_top_acts_indices).tolist()).intersection(set((two_top_acts_indices).tolist()))
# common_feats

In [143]:
len(common_feats)

26

In [144]:
for f_ind in common_feats:
    print(f_ind, feat_labels_lst[f_ind])

3840 phrases related to opportunities and suggestions for improvement
9602 words and themes related to love and affection
8713 phrases that indicate addressing or overcoming challenges
139 references to actions and their consequences, emphasizing occurrences and substances
9362  expressions of affection or appreciation
5525 terms related to assessment and improvement in various contexts
1692 legal and technical terminology related to statutes and inventions
3997 personal reflections on life experiences and relationships
14113 references to historical events and related criticisms of power structures
12967 references to personal interests and passions
9768 terms related to control and authority, particularly in political or systemic contexts
5426  references to collaboration and collective effort in various contexts
13107 expressions of strong negative sentiments or opinions
2230  references to causality and violation of rules or expectations
6983 historical dates and time references
30

In [145]:
nums_only = set((one_top_acts_indices).tolist()) - set((two_top_acts_indices).tolist())
for f_ind in nums_only:
    print(f_ind, feat_labels_lst[f_ind])

11016 prepositions and connections that indicate relationships in the text
12041 elements related to blog interactions and personal reflections
13838 first-person pronouns and related expressions in the text
10640 information about critiques and evaluations in research studies
6292 expressions of strong emotions and personal reflections
1812 details related to business relationships and contracts
14231 phrases focusing on past experiences and reflections on previous content
4892 active verbs related to personal hobbies and outdoor activities
11042 first-person pronouns and expressions of personal experience
6837 terms related to skin conditions and treatments
8630  mentions of cute or appealing items or experiences
10041 instances of adversity and overcoming challenges
4539 references to individuals or entities involved in claims or rumors
9283 fashion-related terms and concepts related to styling
11339 words related to storytelling and narrative elements
9293 content related to reques

In [146]:
months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
for f_ind in months_only:
    print(f_ind, feat_labels_lst[f_ind])

6915 technical terms related to gauge theory and its application in physics
259 mathematical comparisons and relationships, especially involving values and operations
8203  language related to accountability and responsibility
10379 expressions of personal reflection and existential questioning
5517 references to support, community engagement, and social interactions
2841  expressions of disbelief or surprise
6554 negations and phrases indicating absence or exceptions
7971  references to specific scientific articles and researchers
12329  statements about violation of rules or regulations
2604 phrases expressing dislike or hatred towards people or situations.
10156 expressions of refusal or denial
15027 references to personal transformation and significant life experiences
12090 phrases and terms that indicate reduction or simplification of processes and costs
3645 phrases related to measurements or characteristics of products and experiences
6336 terms related to legal and criminal pr

## France v Germany capitals

In [164]:
prompt = "The capital of Washington is Seattle. The capital of France is"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_1 = sae.encode(target_act.to(torch.float32))

In [165]:
prompt = "The capital of Washington is Seattle. The capital of Germany is"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_2 = sae.encode(target_act.to(torch.float32))

In [166]:
sae_acts_1.shape

torch.Size([1, 13, 16384])

### top 15

In [167]:
feat_k = 15
one_top_acts_values, one_top_acts_indices = sae_acts_1[0, -1, :].topk(feat_k, dim=-1)
one_top_acts_indices.sort().values

tensor([ 3022,  3031,  4126,  4314,  4840,  4987,  5486,  5516,  7869,  8651,
         9461,  9768, 12387, 12927, 13414], device='cuda:0')

In [168]:
two_top_acts_values, two_top_acts_indices = sae_acts_2[0, -1, :].topk(feat_k, dim=-1)
two_top_acts_indices.sort().values

tensor([ 3022,  3031,  4314,  4840,  4987,  5486,  8651,  9461,  9768, 12387,
        12927, 13414, 14463, 15316, 16173], device='cuda:0')

In [169]:
common_feats = set((one_top_acts_indices).tolist()).intersection(set((two_top_acts_indices).tolist()))
common_feats

{3022, 3031, 4314, 4840, 4987, 5486, 8651, 9461, 9768, 12387, 12927, 13414}

In [170]:
for f_ind in common_feats:
    print(f_ind, feat_labels_lst[f_ind])

12387 specific locations and associated contexts
13414 statements of existence or presence
9768 terms related to control and authority, particularly in political or systemic contexts
4840 references to historical governance and conflicts in the context of the Abbasid Caliphate
8651 terms related to political institutions and membership
3022 references to geographical locations and their characteristics
5486  interrogative and rhetorical questions related to historical events or entities
9461 references to educational institutions and their locations
3031  references to significant documents or publications, especially in academic or legal contexts
4314 references to churches, bishops, and geographical locations related to religious contexts
4987 references to political figures and their actions
12927 statements of existence or presence


### top 50

In [171]:
feat_k = 50
one_top_acts_values, one_top_acts_indices = sae_acts_1[0, -1, :].topk(feat_k, dim=-1)
# one_top_acts_indices.sort().values

In [172]:
two_top_acts_values, two_top_acts_indices = sae_acts_2[0, -1, :].topk(feat_k, dim=-1)
# two_top_acts_indices.sort().values

In [173]:
for val, ind in zip(one_top_acts_values, one_top_acts_indices):
    print(round(val.item(), 2), ind.item(), feat_labels_lst[ind])

111.58 5516 technical terminology related to scientific research processes
55.76 3031  references to significant documents or publications, especially in academic or legal contexts
45.63 9461 references to educational institutions and their locations
40.83 4314 references to churches, bishops, and geographical locations related to religious contexts
33.46 4987 references to political figures and their actions
27.16 3022 references to geographical locations and their characteristics
26.42 13414 statements of existence or presence
24.91 5486  interrogative and rhetorical questions related to historical events or entities
24.71 4840 references to historical governance and conflicts in the context of the Abbasid Caliphate
21.56 9768 terms related to control and authority, particularly in political or systemic contexts
18.25 8651 terms related to political institutions and membership
18.06 12927 statements of existence or presence
17.3 12387 specific locations and associated contexts
14.71 

In [174]:
for val, ind in zip(two_top_acts_values, two_top_acts_indices):
    print(round(val.item(), 2), ind.item(), feat_labels_lst[ind])

90.84 16173 names of political parties and significant political events
57.17 3031  references to significant documents or publications, especially in academic or legal contexts
56.97 9461 references to educational institutions and their locations
41.64 4314 references to churches, bishops, and geographical locations related to religious contexts
31.25 3022 references to geographical locations and their characteristics
28.53 4987 references to political figures and their actions
25.46 13414 statements of existence or presence
25.03 5486  interrogative and rhetorical questions related to historical events or entities
23.29 9768 terms related to control and authority, particularly in political or systemic contexts
22.78 14463 keywords associated with regional geography and governance
21.98 15316 references to political statements and decisions
21.26 8651 terms related to political institutions and membership
21.0 12387 specific locations and associated contexts
18.86 12927 statements of 

In [175]:
common_feats = set((one_top_acts_indices).tolist()).intersection(set((two_top_acts_indices).tolist()))
# common_feats

In [176]:
len(common_feats)

36

In [177]:
for f_ind in common_feats:
    print(f_ind, feat_labels_lst[f_ind])

259 mathematical comparisons and relationships, especially involving values and operations
6793 elements that resemble structured data or identifiers, likely in a list or JSON format
14489  structures and formats typical in academic or formal writing
1692 legal and technical terminology related to statutes and inventions
4126  information related to numerical expressions and calculations involving prime factors
9120 references to statistical data and comparisons of populations across different countries
9768 terms related to control and authority, particularly in political or systemic contexts
6570 country names and their associations with various metrics or events
8366 verbs and their related forms, often related to medical or technical contexts
3760 terms related to conditions and outcomes in experimental settings
14388 terms and phrases related to technology, particularly in the context of apps, security, and online privacy
1590  comparative phrases that express relationships or con

In [178]:
nums_only = set((one_top_acts_indices).tolist()) - set((two_top_acts_indices).tolist())
for f_ind in nums_only:
    print(f_ind, feat_labels_lst[f_ind])

11010 numerical values, programming symbols, and names of institutions and locations
6306 terms related to African history and governance structures
2021 topics related to political events and elections
6470 references to financial assets and capital-related terms
14155 technical terms and concepts related to medical procedures and research
5516 technical terminology related to scientific research processes
15596 past and present tense forms of the verb "to be" in various contexts
4703 locations and cultural landmarks associated with historical significance
2191 references to individuals involved in historical conflicts and political events
14383 discussions related to political accountability and accusations of malfeasance
15537 terms related to rights and governance
12690 terms related to environmental health and regulations
3645 phrases related to measurements or characteristics of products and experiences
5663 references to Washington, DC and related institutions


In [179]:
months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
for f_ind in months_only:
    print(f_ind, feat_labels_lst[f_ind])

5602 referenced cultural elements or iconic foods, specifically the cheesesteak
15878  references to government structures and regulations, particularly those relating to federal and state authority
1640 references to positions of authority and roles within organizations
9064 mathematical operations and variable assignments in expressions
14463 keywords associated with regional geography and governance
13612  references to luxury car brands and their specific models
16173 names of political parties and significant political events
4750 phrases indicating personal experiences and feelings related to coding and problem-solving
911 various types of definitions and categorizations in a text
308 requests for help or assistance
11445 content related to democracy and governance
15316 references to political statements and decisions
2429 statements about the presence or significance of certain conditions or criteria
10655 terms related to legal processes and court proceedings


## America v China capitals

In [184]:
prompt = "The capital of France is Paris. The capital of America is"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_1 = sae.encode(target_act.to(torch.float32))

In [185]:
prompt = "The capital of France is Paris. The capital of China is"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_2 = sae.encode(target_act.to(torch.float32))

In [186]:
sae_acts_1.shape

torch.Size([1, 13, 16384])

### top 15

In [187]:
feat_k = 15
one_top_acts_values, one_top_acts_indices = sae_acts_1[0, -1, :].topk(feat_k, dim=-1)
one_top_acts_indices.sort().values

tensor([ 1583,  3022,  3031,  4314,  4840,  4987,  5486,  5663,  8366,  9461,
         9768, 12927, 13414, 15596, 16033], device='cuda:0')

In [188]:
two_top_acts_values, two_top_acts_indices = sae_acts_2[0, -1, :].topk(feat_k, dim=-1)
two_top_acts_indices.sort().values

tensor([ 3022,  3031,  3242,  3915,  4314,  4840,  4987,  5486,  8366,  9461,
         9768, 11549, 12927, 13414, 13737], device='cuda:0')

In [189]:
common_feats = set((one_top_acts_indices).tolist()).intersection(set((two_top_acts_indices).tolist()))
common_feats

{3022, 3031, 4314, 4840, 4987, 5486, 8366, 9461, 9768, 12927, 13414}

In [190]:
for f_ind in common_feats:
    print(f_ind, feat_labels_lst[f_ind])

13414 statements of existence or presence
9768 terms related to control and authority, particularly in political or systemic contexts
4840 references to historical governance and conflicts in the context of the Abbasid Caliphate
5486  interrogative and rhetorical questions related to historical events or entities
3022 references to geographical locations and their characteristics
8366 verbs and their related forms, often related to medical or technical contexts
9461 references to educational institutions and their locations
3031  references to significant documents or publications, especially in academic or legal contexts
4314 references to churches, bishops, and geographical locations related to religious contexts
4987 references to political figures and their actions
12927 statements of existence or presence


### top 50

In [191]:
feat_k = 50
one_top_acts_values, one_top_acts_indices = sae_acts_1[0, -1, :].topk(feat_k, dim=-1)
# one_top_acts_indices.sort().values

In [192]:
two_top_acts_values, two_top_acts_indices = sae_acts_2[0, -1, :].topk(feat_k, dim=-1)
# two_top_acts_indices.sort().values

In [193]:
for val, ind in zip(one_top_acts_values, one_top_acts_indices):
    print(round(val.item(), 2), ind.item(), feat_labels_lst[ind])

68.53 3031  references to significant documents or publications, especially in academic or legal contexts
61.96 5663 references to Washington, DC and related institutions
55.89 16033 mentions of U.S. entities or actions, particularly those related to government and economics
37.14 3022 references to geographical locations and their characteristics
33.87 5486  interrogative and rhetorical questions related to historical events or entities
32.35 13414 statements of existence or presence
26.32 4987 references to political figures and their actions
25.19 9461 references to educational institutions and their locations
24.13 9768 terms related to control and authority, particularly in political or systemic contexts
22.01 12927 statements of existence or presence
21.3 8366 verbs and their related forms, often related to medical or technical contexts
20.94 4314 references to churches, bishops, and geographical locations related to religious contexts
20.18 1583 words related to administrative o

In [194]:
for val, ind in zip(two_top_acts_values, two_top_acts_indices):
    print(round(val.item(), 2), ind.item(), feat_labels_lst[ind])

84.65 3242 information related to corruption and bribery involving public officials
58.14 9461 references to educational institutions and their locations
54.55 3031  references to significant documents or publications, especially in academic or legal contexts
43.04 4314 references to churches, bishops, and geographical locations related to religious contexts
36.21 3022 references to geographical locations and their characteristics
31.77 5486  interrogative and rhetorical questions related to historical events or entities
29.57 13414 statements of existence or presence
24.43 3915 terms related to criticism of political ideologies and figures
22.57 9768 terms related to control and authority, particularly in political or systemic contexts
21.29 11549  references to specific Asian cuisines, particularly Chinese dishes and ingredients
20.97 4840 references to historical governance and conflicts in the context of the Abbasid Caliphate
20.24 4987 references to political figures and their act

In [195]:
common_feats = set((one_top_acts_indices).tolist()).intersection(set((two_top_acts_indices).tolist()))
# common_feats

In [196]:
len(common_feats)

34

In [197]:
for f_ind in common_feats:
    print(f_ind, feat_labels_lst[f_ind])

15878  references to government structures and regulations, particularly those relating to federal and state authority
6793 elements that resemble structured data or identifiers, likely in a list or JSON format
4750 phrases indicating personal experiences and feelings related to coding and problem-solving
6163 monetary values and their associated attributes in different contexts
14489  structures and formats typical in academic or formal writing
1692 legal and technical terminology related to statutes and inventions
4126  information related to numerical expressions and calculations involving prime factors
10655 terms related to legal processes and court proceedings
9768 terms related to control and authority, particularly in political or systemic contexts
6570 country names and their associations with various metrics or events
8366 verbs and their related forms, often related to medical or technical contexts
1583 words related to administrative or technical tasks and issues
1590  comp

In [198]:
nums_only = set((one_top_acts_indices).tolist()) - set((two_top_acts_indices).tolist())
for f_ind in nums_only:
    print(f_ind, feat_labels_lst[f_ind])

8096 references to the city of Washington, D.C. and its variations
16033 mentions of U.S. entities or actions, particularly those related to government and economics
2626 legal terminology related to regulatory definitions and company classifications
259 mathematical comparisons and relationships, especially involving values and operations
7876 elements related to ratings and classifications
16225 references to political parties and issues involving the Republican Party
13668 discussions about political norms and behaviors
12935 questions related to economic efficiency, fairness, and environmental impact
6631 the beginning of a text or important markers in a document
615 terms related to legal proceedings and political processes
1640 references to positions of authority and roles within organizations
5663 references to Washington, DC and related institutions
2964  phrases that indicate relationships or connections involving the concept of "to" and associated actions
308 requests for he

In [199]:
months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
for f_ind in months_only:
    print(f_ind, feat_labels_lst[f_ind])

11010 numerical values, programming symbols, and names of institutions and locations
3645 phrases related to measurements or characteristics of products and experiences
14819 references to geopolitical tensions, particularly involving Iran, Russia, and nuclear issues
14023  references to financial transactions and their management
13737 specific names or titles, particularly related to Mongolian fighters or historical figures
3242 information related to corruption and bribery involving public officials
523 descriptions of geographical locations and administrative divisions
3915 terms related to criticism of political ideologies and figures
9419 names of individuals and titles
3760 terms related to conditions and outcomes in experimental settings
15728 elements related to injury and trauma
7700 references to historical figures and events, particularly relating to royalty
3956 references to geographic locations and landmarks
6260 dialogue involving questions and uncertainties about ident

# find common features (batch)

## nums vs months

In [39]:
words = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve']
numword_samps = []
for i in range(0, 9):
    samp = f"{words[i]} {words[i+1]} {words[i+2]} {words[i+3]}"
    numword_samps.append(samp)

In [19]:
numword_samps

['one two three four',
 'two three four five',
 'three four five six',
 'four five six seven',
 'five six seven eight',
 'six seven eight nine',
 'seven eight nine ten',
 'eight nine ten eleven',
 'nine ten eleven twelve']

In [40]:
# batch_tokens
inputs = tokenizer(numword_samps, return_tensors="pt", padding=True, truncation=True, max_length=300)['input_ids'].to("cuda")
inputs.shape

torch.Size([9, 5])

In [21]:
inputs

tensor([[    2,   785,  1378,  2149,  2785],
        [    2,  9721,  2149,  2785,  4105],
        [    2, 16183,  2785,  4105,  4442],
        [    2, 14806,  4105,  4442,  6861],
        [    2, 17159,  4442,  6861,  8469],
        [    2, 22789,  6861,  8469,  9981],
        [    2, 31033,  8469,  9981,  2797],
        [    2, 40363,  9981,  2797, 29754],
        [    2, 37527,  2797, 29754, 20779]])

In [41]:
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_1 = sae.encode(target_act.to(torch.float32))

In [37]:
words = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
month_samps = []
for i in range(0, 9):
    samp = f"{words[i]} {words[i+1]} {words[i+2]} {words[i+3]}"
    month_samps.append(samp)
inputs = tokenizer(month_samps, return_tensors="pt", padding=True, truncation=True, max_length=300)['input_ids'].to("cuda")
inputs.shape

torch.Size([9, 5])

In [38]:
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_2 = sae.encode(target_act.to(torch.float32))

### top 15

In [89]:
feat_k = 15
one_top_acts_values, one_top_acts_indices = sae_acts_1[:, -1, :].topk(feat_k, dim=-1)
one_top_acts_indices.sort().values

tensor([[ 2269,  3019,  3109,  5441,  6131,  6300,  6631,  8640,  9768, 10230,
         10802, 10841, 11864, 14441, 15451],
        [ 2740,  3019,  3109,  5441,  6167,  6305,  6631,  8737,  9768, 10230,
         10841, 11003, 12802, 14441, 15782],
        [ 2102,  3019,  5441,  6131,  6305,  6631,  8242,  9768, 10230, 10841,
         11003, 11771, 14441, 14471, 15735],
        [ 3019,  5441,  5676,  6131,  6143,  9768, 10230, 10841, 10973, 11003,
         12658, 12977, 14441, 14898, 15782],
        [ 1094,  3032,  3435,  4743,  5441,  6131,  8211,  9768,  9785, 10230,
         10841, 10973, 12457, 12658, 14441],
        [ 1316,  1862,  5441,  6131,  6143,  6567,  8366,  9768, 10052, 10230,
         10615, 10973, 12457, 12658, 13206],
        [  855,  1862,  5441,  6131,  6143,  7709,  8366,  9768, 10230, 10615,
         10973, 11674, 13908, 15704, 16271],
        [  827,  1862,  3697,  5441,  6131,  6143,  6305,  6618,  8366,  9768,
         10230, 10738, 12161, 15782, 16271],
        

In [90]:
one_top_acts_values

tensor([[64.4877, 45.1703, 40.5895, 35.3551, 34.3690, 30.3312, 30.2256, 21.9555,
         21.9007, 21.7239, 20.6970, 20.2885, 19.6938, 19.0292, 18.0701],
        [60.3583, 43.7196, 42.4637, 40.5117, 37.9534, 37.7351, 24.6161, 21.6346,
         21.3586, 21.2357, 20.6309, 19.9925, 18.8740, 18.5112, 18.2505],
        [51.6413, 45.3826, 39.5861, 37.4597, 32.6303, 29.5705, 25.5231, 24.9491,
         23.3405, 22.6388, 21.8672, 21.5575, 21.5400, 21.1547, 20.9332],
        [76.0729, 48.0570, 44.7954, 40.7604, 35.5247, 26.0187, 24.7982, 23.2534,
         22.9992, 22.3749, 19.4121, 19.1158, 18.9582, 18.2932, 17.8383],
        [84.7398, 37.8657, 34.1638, 30.8324, 30.5867, 28.2125, 24.2328, 23.0589,
         21.0128, 20.7138, 20.6023, 19.1343, 18.4614, 17.7337, 16.4010],
        [40.7494, 39.8484, 37.5722, 36.8509, 36.0316, 33.7934, 31.9825, 29.7316,
         26.6292, 24.2043, 23.8879, 21.7214, 21.7174, 20.2330, 19.3348],
        [59.3423, 56.7082, 38.3538, 32.4716, 29.7215, 27.0327, 26.6668, 25.7

In [91]:
two_top_acts_values, two_top_acts_indices = sae_acts_2[:, -1, :].topk(feat_k, dim=-1)
two_top_acts_indices.sort().values

tensor([[  522,   609,  1530,  2269,  3109,  3162,  5710,  5968,  6240,  7159,
          9768, 11546, 12748, 13767, 15451],
        [  609,  1094,  1530,  3109,  3162,  5710,  6167,  6240,  7159,  7985,
          8737,  9768, 10636, 11546, 12802],
        [  609,  1094,  1530,  3162,  6240,  7159,  8242,  9768, 10103, 10636,
         11546, 12920, 14471, 15715, 15735],
        [  609,  1094,  3162,  5404,  5676,  6240,  7159,  9768, 10103, 10973,
         11004, 11546, 12161, 12977, 15763],
        [  609,  1094,  2762,  3162,  5404,  6240,  7159,  8211,  9768,  9785,
         10636, 11546, 12161, 12658, 14017],
        [  609,  1094,  1316,  1862,  3162,  4295,  4997,  5404,  6240,  7159,
          9768, 10052, 10636, 11546, 12161],
        [  609,  1094,  1862,  3162,  5404,  5968,  6240,  7159,  7709,  9768,
         10327, 10636, 11546, 12161, 13908],
        [  609,  1094,  3162,  5900,  6240,  6305,  6618,  7159,  8506,  9768,
         10636, 11546, 12161, 12802, 14540],
        

In [92]:
# common_feats = set((one_top_acts_indices).tolist()).intersection(set((two_top_acts_indices).tolist()))
# Assuming one_top_acts_indices and two_top_acts_indices are tensors obtained similarly
one_top_acts_indices_list = one_top_acts_indices.tolist()
two_top_acts_indices_list = two_top_acts_indices.tolist()

# Calculate the intersection of indices for each corresponding row
common_feats_per_row = [
    list(set(row_one).intersection(set(row_two)))
    for row_one, row_two in zip(one_top_acts_indices_list, two_top_acts_indices_list)
]

common_feats_per_row

[[9768, 2269, 15451, 3109],
 [8737, 12802, 3109, 9768, 6167],
 [9768, 8242, 15735, 14471],
 [9768, 12977, 5676, 10973],
 [1094, 9768, 12658, 8211, 9785],
 [9768, 1316, 1862, 10052],
 [9768, 13908, 7709, 1862],
 [9768, 6305, 12161, 6618],
 [9768, 6305, 6618]]

In [93]:
if common_feats_per_row:  # Check if the list is not empty
    # Start with the first set in the list
    common_in_all_rows = set(common_feats_per_row[0])

    # Intersect with each subsequent set in the list
    for common_set in common_feats_per_row[1:]:
        common_in_all_rows.intersection_update(set(common_set))
else:
    common_in_all_rows = set()

common_in_all_rows

{9768}

In [94]:
for sampID, samp_common_feats in enumerate(common_feats_per_row):
    print('samp: ', sampID)
    for f_ind in samp_common_feats:
        print(f_ind, feat_labels_lst[f_ind])

samp:  0
9768 terms related to control and authority, particularly in political or systemic contexts
2269  structured lists or numbered steps in procedural texts
15451  structured data or attributes in a coded format
3109  technical terms and symbols related to data structures
samp:  1
8737  elements related to numerical values or ordered data
12802  numerical values and codes typically associated with scientific or technical contexts
3109  technical terms and symbols related to data structures
9768 terms related to control and authority, particularly in political or systemic contexts
6167  structured information and mathematical relationships
samp:  2
9768 terms related to control and authority, particularly in political or systemic contexts
8242 references and citations in scientific writing
15735  numerical data and comparisons
14471 specific numerical values and their formatting
samp:  3
9768 terms related to control and authority, particularly in political or systemic contexts
129

In [95]:
if common_feats_per_row:  # Check if the list is not empty
    # Start with the first set in the list
    common_in_all_rows = set(common_feats_per_row[0])

    # Intersect with each subsequent set in the list
    for common_set in common_feats_per_row[1:]:
        common_in_all_rows.intersection_update(set(common_set))
else:
    common_in_all_rows = set()

common_in_all_rows

{9768}

In [96]:
# for sampID, samp_common_feats in enumerate(common_feats_per_row):
#     print('samp: ', sampID)
#     for f_ind in samp_common_feats[:3]:
#         print(f_ind, feat_labels_lst[f_ind])

In [97]:
# for val, ind in zip(two_top_acts_values[1], two_top_acts_indices[1]):
#     print(round(val.item(), 2), ind.item(), feat_labels_lst[ind])

In [98]:
common_feats_all = list(set([index for sublist in common_feats_per_row for index in sublist]))
len(common_feats_all)

25

In [99]:
num_feats_all = list(set(one_top_acts_indices.flatten().tolist()))
len(num_feats_all)

64

In [100]:
month_feats_all = list(set(two_top_acts_indices.flatten().tolist()))
len(month_feats_all)

56

In [101]:
nums_only = set(num_feats_all) - set(month_feats_all)
len(nums_only)
# for f_ind in nums_only:
#     print(f_ind, feat_labels_lst[f_ind])

38

In [102]:
months_only = set(month_feats_all) - set(num_feats_all)
len(months_only)
# for f_ind in months_only:
#     print(f_ind, feat_labels_lst[f_ind])

30

### top 50

In [55]:
feat_k = 50
one_top_acts_values, one_top_acts_indices = sae_acts_1[:, -1, :].topk(feat_k, dim=-1)

In [56]:
two_top_acts_values, two_top_acts_indices = sae_acts_2[:, -1, :].topk(feat_k, dim=-1)

In [ ]:
one_top_acts_indices_list = one_top_acts_indices.tolist()
two_top_acts_indices_list = two_top_acts_indices.tolist()
common_feats_per_row = [
    list(set(row_one).intersection(set(row_two)))
    for row_one, row_two in zip(one_top_acts_indices_list, two_top_acts_indices_list)
]

In [74]:
len(common_feats_per_row[2])

16

In [58]:
if common_feats_per_row:  # Check if the list is not empty
    # Start with the first set in the list
    common_in_all_rows = set(common_feats_per_row[0])

    # Intersect with each subsequent set in the list
    for common_set in common_feats_per_row[1:]:
        common_in_all_rows.intersection_update(set(common_set))
else:
    common_in_all_rows = set()

common_in_all_rows

{3019, 9768, 12658}

In [ ]:
# for sampID, samp_common_feats in enumerate(common_feats_per_row):
#     print('samp: ', sampID)
#     for f_ind in samp_common_feats[:3]:
#         print(f_ind, feat_labels_lst[f_ind])

In [ ]:
# for val, ind in zip(two_top_acts_values[1], two_top_acts_indices[1]):
#     print(round(val.item(), 2), ind.item(), feat_labels_lst[ind])

In [80]:
common_feats_all = list(set([index for sublist in common_feats_per_row for index in sublist]))
len(common_feats_all)

58

In [67]:
num_feats_all = list(set(one_top_acts_indices.flatten().tolist()))
len(num_feats_all)

155

In [66]:
month_feats_all = list(set(two_top_acts_indices.flatten().tolist()))
len(month_feats_all)

154

In [69]:
nums_only = set(num_feats_all) - set(month_feats_all)
len(nums_only)
# for f_ind in nums_only:
#     print(f_ind, feat_labels_lst[f_ind])

95

In [70]:
months_only = set(month_feats_all) - set(num_feats_all)
len(months_only)
# for f_ind in months_only:
#     print(f_ind, feat_labels_lst[f_ind])

94

# nums v months umap on feature subset

In [82]:
import umap

## top 15

In [103]:
feature_subset = num_feats_all + month_feats_all
feature_subset = list(set(feature_subset))
feature_subset.sort()
len(feature_subset)

94

In [104]:
f_labels = [ "F" + str(f_ind) + " : " + feat_labels_lst[f_ind] for f_ind in feature_subset]

Now labels and weight matrix may share the same new indices (rather than feature numbers of orig matrix) due to slicing them in same way

In [105]:
weights_feature_subset = weight_matrix[feature_subset]
weights_feature_subset.shape

(94, 2304)

In [106]:
reducer = umap.UMAP(n_neighbors=15, min_dist=0.01, metric='euclidean')
embedding1 = reducer.fit_transform(weights_feature_subset)

In [108]:
common_feats = common_feats_all
# months_only =

In [109]:
import plotly.express as px
import numpy as np
import pandas as pd

df = pd.DataFrame(embedding1, columns=['UMAP Component 1', 'UMAP Component 2'])
df['Feature ID'] = feature_subset
# df['Feature Description']
df['feature'] = [i[:75] for i in f_labels] # [str(i) for i in feature_subset] # feature_subset
df['Color'] = df['Feature ID'].apply(lambda x: 'green' if x in common_feats else ('red' if x in months_only else 'blue'))

fig = px.scatter(df, x='UMAP Component 1', y='UMAP Component 2', # text='Feature ID',
                 color='Color',  # Use the Color column for coloring points
                 color_discrete_map={'red': 'red', 'green': 'green', 'blue': 'blue'},
                #  hover_data='Feature Description')
                hover_data={
                     'UMAP Component 1': False,
                     'UMAP Component 2': False,
                     'Feature ID': False,
                     'Color': False,
                     'feature': True  # Only display the feature description
                 } ) #,
                #  size=[0.1] * len(df))

fig.update_layout(
    title='UMAP of Decoder Weights',
    xaxis_title='UMAP Component 1',
    yaxis_title='UMAP Component 2'
)

# Customize legend labels
fig.for_each_trace(lambda t: t.update(name = {
    'red': 'Months Only',
    'green': 'Both',
    'blue': 'Numbers Only'
}[t.name]))

fig.update_traces(marker=dict(size= 8))

fig.show()

In [ ]:
from google.colab import files
output_filename = 'umap.html'
fig.write_html(output_filename)
files.download(output_filename)

## top 50

In [83]:
feature_subset = num_feats_all + month_feats_all
feature_subset = list(set(feature_subset))
feature_subset.sort()
len(feature_subset)

249

In [84]:
f_labels = [ "F" + str(f_ind) + " : " + feat_labels_lst[f_ind] for f_ind in feature_subset]

Now labels and weight matrix may share the same new indices (rather than feature numbers of orig matrix) due to slicing them in same way

In [85]:
weights_feature_subset = weight_matrix[feature_subset]
weights_feature_subset.shape

(249, 2304)

In [86]:
reducer = umap.UMAP(n_neighbors=15, min_dist=0.01, metric='euclidean')
embedding1 = reducer.fit_transform(weights_feature_subset)

In [87]:
common_feats = common_feats_all
# months_only =

In [88]:
import plotly.express as px
import numpy as np
import pandas as pd

df = pd.DataFrame(embedding1, columns=['UMAP Component 1', 'UMAP Component 2'])
df['Feature ID'] = feature_subset
# df['Feature Description']
df['feature'] = [i[:75] for i in f_labels] # [str(i) for i in feature_subset] # feature_subset
df['Color'] = df['Feature ID'].apply(lambda x: 'green' if x in common_feats else ('red' if x in months_only else 'blue'))

fig = px.scatter(df, x='UMAP Component 1', y='UMAP Component 2', # text='Feature ID',
                 color='Color',  # Use the Color column for coloring points
                 color_discrete_map={'red': 'red', 'green': 'green', 'blue': 'blue'},
                #  hover_data='Feature Description')
                hover_data={
                     'UMAP Component 1': False,
                     'UMAP Component 2': False,
                     'Feature ID': False,
                     'Color': False,
                     'feature': True  # Only display the feature description
                 } ) #,
                #  size=[0.1] * len(df))

fig.update_layout(
    title='UMAP of Decoder Weights',
    xaxis_title='UMAP Component 1',
    yaxis_title='UMAP Component 2'
)

# Customize legend labels
fig.for_each_trace(lambda t: t.update(name = {
    'red': 'Months Only',
    'green': 'Both',
    'blue': 'Numbers Only'
}[t.name]))

fig.update_traces(marker=dict(size= 8))

fig.show()

In [ ]:
from google.colab import files
output_filename = 'umap.html'
fig.write_html(output_filename)
files.download(output_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# I love v I hate umap on feature subset

top 50

In [154]:
feature_subset = two_top_acts_indices.tolist() + one_top_acts_indices.tolist()
feature_subset = list(set(feature_subset))
feature_subset.sort()
len(feature_subset)

74

In [155]:
f_labels = [ "F" + str(f_ind) + " : " + feat_labels_lst[f_ind] for f_ind in feature_subset]

Now labels and weight matrix may share the same new indices (rather than feature numbers of orig matrix) due to slicing them in same way

In [156]:
weights_feature_subset = weight_matrix[feature_subset]
weights_feature_subset.shape

(74, 2304)

In [157]:
reducer = umap.UMAP(n_neighbors=15, min_dist=0.01, metric='euclidean')
embedding1 = reducer.fit_transform(weights_feature_subset)

In [158]:
common_feats = set((one_top_acts_indices).tolist()).intersection(set((two_top_acts_indices).tolist()))
months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())

In [159]:
common_feats = list(common_feats)
months_only = list(months_only )

In [160]:
import plotly.express as px
import numpy as np
import pandas as pd

df = pd.DataFrame(embedding1, columns=['UMAP Component 1', 'UMAP Component 2'])
df['Feature ID'] = feature_subset
# df['Feature Description']
df['feature'] = [i[:75] for i in f_labels] # [str(i) for i in feature_subset] # feature_subset
df['Color'] = df['Feature ID'].apply(lambda x: 'green' if x in common_feats else ('red' if x in months_only else 'blue'))

fig = px.scatter(df, x='UMAP Component 1', y='UMAP Component 2', # text='Feature ID',
                 color='Color',  # Use the Color column for coloring points
                 color_discrete_map={'red': 'red', 'green': 'green', 'blue': 'blue'},
                #  hover_data='Feature Description')
                hover_data={
                     'UMAP Component 1': False,
                     'UMAP Component 2': False,
                     'Feature ID': False,
                     'Color': False,
                     'feature': True  # Only display the feature description
                 } ) #,
                #  size=[0.1] * len(df))

fig.update_layout(
    title='UMAP of Decoder Weights',
    xaxis_title='UMAP Component 1',
    yaxis_title='UMAP Component 2'
)

# Customize legend labels
fig.for_each_trace(lambda t: t.update(name = {
    'red': 'I hate',
    'green': 'Both',
    'blue': 'I love'
}[t.name]))

fig.update_traces(marker=dict(size= 8))

fig.show()

# America v China umap on feature subset

top 50

In [200]:
feature_subset = two_top_acts_indices.tolist() + one_top_acts_indices.tolist()
feature_subset = list(set(feature_subset))
feature_subset.sort()
len(feature_subset)

66

In [201]:
f_labels = [ "F" + str(f_ind) + " : " + feat_labels_lst[f_ind] for f_ind in feature_subset]

Now labels and weight matrix may share the same new indices (rather than feature numbers of orig matrix) due to slicing them in same way

In [202]:
weights_feature_subset = weight_matrix[feature_subset]
weights_feature_subset.shape

(66, 2304)

In [203]:
reducer = umap.UMAP(n_neighbors=15, min_dist=0.01, metric='euclidean')
embedding1 = reducer.fit_transform(weights_feature_subset)

In [204]:
common_feats = set((one_top_acts_indices).tolist()).intersection(set((two_top_acts_indices).tolist()))
months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())

In [205]:
common_feats = list(common_feats)
months_only = list(months_only )

In [206]:
import plotly.express as px
import numpy as np
import pandas as pd

df = pd.DataFrame(embedding1, columns=['UMAP Component 1', 'UMAP Component 2'])
df['Feature ID'] = feature_subset
# df['Feature Description']
df['feature'] = [i[:75] for i in f_labels] # [str(i) for i in feature_subset] # feature_subset
df['Color'] = df['Feature ID'].apply(lambda x: 'green' if x in common_feats else ('red' if x in months_only else 'blue'))

fig = px.scatter(df, x='UMAP Component 1', y='UMAP Component 2', # text='Feature ID',
                 color='Color',  # Use the Color column for coloring points
                 color_discrete_map={'red': 'red', 'green': 'green', 'blue': 'blue'},
                #  hover_data='Feature Description')
                hover_data={
                     'UMAP Component 1': False,
                     'UMAP Component 2': False,
                     'Feature ID': False,
                     'Color': False,
                     'feature': True  # Only display the feature description
                 } ) #,
                #  size=[0.1] * len(df))

fig.update_layout(
    title='UMAP of Decoder Weights',
    xaxis_title='UMAP Component 1',
    yaxis_title='UMAP Component 2'
)

# Customize legend labels
fig.for_each_trace(lambda t: t.update(name = {
    'red': 'I hate',
    'green': 'Both',
    'blue': 'I love'
}[t.name]))

fig.update_traces(marker=dict(size= 8))

fig.show()

# ablate then generate

## patch using transformerlens

In [ ]:
%%capture
!pip install transformer_lens

In [ ]:
from transformer_lens import HookedTransformer

# uses a lot of memory, use A100
model_2 = HookedTransformer.from_pretrained(
    "gemma-2-2b"
)

config.json:   0%|          | 0.00/818 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Loaded pretrained model gemma-2-2b into HookedTransformer


In [ ]:
prompt = "January February March April"
tokens = model_2.to_tokens(prompt).to(device)
logits = model_2(tokens)

next_token = logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

' May'

In [ ]:
model_2.blocks[20].hook_resid_post

HookPoint()

In [ ]:
LLM_patch = torch.zeros(1, 5, 2304)

In [ ]:
# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch=LLM_patch,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

In [ ]:
next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

'<pad>'

## patch one feature

In [ ]:
prompt = "January February March April"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_2 = sae.encode(target_act.to(torch.float32))

In [ ]:
months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
for f_ind in list(months_only)[:5]:
    print(f_ind, feat_labels_lst[f_ind])

12161 terms related to programming and technical constructs
10636 specific dates or references to time
10254 references to days of the week and their associated schedules
1039 technical terms and elements within programming and coding contexts
15763  sequences of numbers and their occurrences within the document


In [ ]:
sae_acts_2[0, -1, 12161]

tensor(11.3316, device='cuda:0')

In [ ]:
# ablate a feature (idx = 12161) by setting it to 0
sae_acts_2[0, -1, 12161] = 0
sae_acts_2[0, -1, 12161]

tensor(0., device='cuda:0')

In [ ]:
recon = sae.decode(sae_acts_2)
recon.shape

torch.Size([1, 5, 2304])

In [ ]:
# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch= recon,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

In [ ]:
next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

' May'

## patch all month only features

In [ ]:
prompt = "January February March April"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_2 = sae.encode(target_act.to(torch.float32))

In [ ]:
months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
for f_ind in list(months_only):
    sae_acts_2[0, -1, f_ind] = 0
recon = sae.decode(sae_acts_2)
recon.shape

torch.Size([1, 5, 2304])

In [ ]:
# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch= recon,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

' '

## patch half month only features




In [ ]:
prompt = "January February March April"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_2 = sae.encode(target_act.to(torch.float32))

In [ ]:
months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
for f_ind in list(months_only)[:25]:
    sae_acts_2[0, -1, f_ind] = 0
recon = sae.decode(sae_acts_2)
recon.shape

torch.Size([1, 5, 2304])

In [ ]:
# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch= recon,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

'\n'

## gen more than next token

In [ ]:
prompt = "January February March April"
clean_text = prompt
tokens = model_2.to_tokens(prompt).to(device)

for i in range(3):
    if next_char == '':
        next_char = ' '

    clean_text = clean_text + next_char

    # tokens = torch.cat([tokens, next_token[None, None]], dim=-1)
    tokens = model_2.to_tokens(clean_text).to(device)

    #########
    inputs = tokenizer.encode(clean_text, return_tensors="pt", add_special_tokens=True).to("cuda")
    target_act = gather_residual_activations(model, 20, inputs)
    sae_acts_2 = sae.encode(target_act.to(torch.float32))

    for f_ind in list(months_only)[:25]:
        sae_acts_2[0, -1, f_ind] = 0
    recon = sae.decode(sae_acts_2)

    #########
    def patch_layer(
        orig_actvs: Float[Tensor, "batch pos d_model"],
        hook: HookPoint,
        LLM_patch: Float[Tensor, "batch pos d_model"],
        layer_to_patch: int,
    ) -> Float[Tensor, "batch pos d_model"]:
        if layer_to_patch == hook.layer():
            orig_actvs[:, :, :] = LLM_patch
        return orig_actvs

    hook_fn = partial(
            patch_layer,
            LLM_patch= recon,
            layer_to_patch = 20
        )

    #########
    ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )
    next_token = ablated_logits[0, -1].argmax(dim=-1) # Get the predicted token at the end of our sequence
    # next_char = model.to_string(next_token)

print(model_2.to_string(tokens))

['<bos>January February March April\n\n\n']


## patch top features




around 23 to destroy

In [ ]:
# for val, ind in zip(two_top_acts_values, two_top_acts_indices):
#     print(round(val.item(), 2), ind.item(), feat_labels_lst[ind])

65.1 15451  structured data or attributes in a coded format
41.55 5710 references to seasonal festivals and their associated cultural significance
38.81 13767 terms related to approval and assessment in a regulatory or medical context
34.97 11546 dates and months
33.7 6240  words related to time indicators, specifically months or dates
33.65 9768 terms related to control and authority, particularly in political or systemic contexts
32.96 3109  technical terms and symbols related to data structures
28.4 2269  structured lists or numbered steps in procedural texts
27.97 7159  references to time frames and chronological events
27.64 3162  markers indicating the beginning of a document
24.91 609 terms related to research articles and their attributes, specifically focusing on methodologies and data contributions
24.85 12748  structured data representations and their attributes
23.26 5968 references to specific dates, particularly in October and April
22.92 522  numerical data related to ge

In [ ]:
prompt = "January February March April"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_2 = sae.encode(target_act.to(torch.float32))

In [ ]:
tokens = model_2.to_tokens(prompt).to(device)

In [ ]:
top_months_only_f = [feat for feat in list(months_only) if feat in two_top_acts_indices[:9] ]

In [ ]:
two_top_acts_indices[:9]

tensor([15451,  5710, 13767, 11546,  6240,  9768,  3109,  2269,  7159],
       device='cuda:0')

In [ ]:
len(months_only)

38

In [ ]:
# months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
# for f_ind in list(months_only)[:23]:
for f_ind in top_months_only_f:
    sae_acts_2[0, -1, f_ind] = 0
recon = sae.decode(sae_acts_2)
recon.shape

torch.Size([1, 5, 2304])

In [ ]:
# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch= recon,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

'\n'

In [ ]:
ablated_logits[0, -1].argmax(dim=-1)

tensor(108, device='cuda:0')

In [ ]:
ablated_logits[0, -1, 2782]

tensor(14.3767, device='cuda:0')

In [ ]:
ablated_logits[0, -1, 108]

tensor(15.3885, device='cuda:0')

In [ ]:
sae_acts_2 = sae.encode(target_act.to(torch.float32))
# top_months_only_f = [feat for feat in list(months_only) if feat in two_top_acts_indices[:9] ]
# for f_ind in top_months_only_f:
sae_acts_2[0, -1, 7159] = 0
recon = sae.decode(sae_acts_2)
recon.shape

# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch= recon,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

' May'

In [ ]:
ablated_logits[0, -1].argmax(dim=-1)

tensor(2782, device='cuda:0')

In [ ]:
ablated_logits[0, -1, 2782]

tensor(17.3316, device='cuda:0')

In [ ]:
prompt = "January February March April"
clean_text = prompt
tokens = model_2.to_tokens(prompt).to(device)
next_char = 0

top_months_only_f = [feat for feat in list(months_only) if feat in two_top_acts_indices[:1] ]

for i in range(10):

    #########
    inputs = tokenizer.encode(clean_text, return_tensors="pt", add_special_tokens=True).to("cuda")
    target_act = gather_residual_activations(model, 20, inputs)
    sae_acts_2 = sae.encode(target_act.to(torch.float32))

    # for f_ind in top_months_only_f:
    #     sae_acts_2[0, -1, f_ind] = 0
    recon = sae.decode(sae_acts_2)

    #########
    def patch_layer(
        orig_actvs: Float[Tensor, "batch pos d_model"],
        hook: HookPoint,
        LLM_patch: Float[Tensor, "batch pos d_model"],
        layer_to_patch: int,
    ) -> Float[Tensor, "batch pos d_model"]:
        if layer_to_patch == hook.layer():
            orig_actvs[:, :, :] = LLM_patch
        return orig_actvs

    hook_fn = partial(
            patch_layer,
            LLM_patch= recon,
            layer_to_patch = 20
        )

    #########
    ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )
    next_token = ablated_logits[0, -1].argmax(dim=-1) # Get the predicted token at the end of our sequence
    next_char = model_2.to_string(next_token)
    print("Next char: ", next_char)
    print(model_2.to_string(tokens))

    if next_char == '':
        next_char = ' '

    clean_text = clean_text + next_char

    print(clean_text)

    # tokens = torch.cat([tokens, next_token[None, None]], dim=-1)
    tokens = model_2.to_tokens(clean_text).to(device)

print(model_2.to_string(tokens))

Next char:   May
['<bos>January February March April']
January February March April May
Next char:  

['<bos>January February March April May']
January February March April May

Next char:   
['<bos>January February March April May\n']
January February March April May
 
Next char:  

['<bos>January February March April May\n ']
January February March April May
 

Next char:   
['<bos>January February March April May\n \n']
January February March April May
 
 
Next char:  

['<bos>January February March April May\n \n ']
January February March April May
 
 

Next char:   
['<bos>January February March April May\n \n \n']
January February March April May
 
 
 
Next char:  

['<bos>January February March April May\n \n \n ']
January February March April May
 
 
 

Next char:   
['<bos>January February March April May\n \n \n \n']
January February March April May
 
 
 
 
Next char:  

['<bos>January February March April May\n \n \n \n ']
January February March April May
 
 
 
 

['<bos>Jan

# steer nums to months

In [ ]:
prompt = "one two three four"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_1 = sae.encode(target_act.to(torch.float32))
recon = sae.decode(sae_acts_1)

tokens = model_2.to_tokens(prompt).to(device)

# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch= recon,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

' five'

In [ ]:
prompt = "one two three four"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_1 = sae.encode(target_act.to(torch.float32))

In [ ]:
prompt = "January February March April"
inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_2 = sae.encode(target_act.to(torch.float32))

In [ ]:
months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
for f_ind in list(months_only):
    sae_acts_1[0, -1, f_ind] = sae_acts_2[0, -1, f_ind]
recon = sae.decode(sae_acts_1)
recon.shape

torch.Size([1, 5, 2304])

In [ ]:
for f_ind in list(months_only):
    print(sae_acts_1[0, -1, f_ind])
    print(sae_acts_1[0, -1, f_ind] - sae_acts_2[0, -1, f_ind])

tensor(7.8716, device='cuda:0')
tensor(-3.4600, device='cuda:0')
tensor(0., device='cuda:0')
tensor(-19.8431, device='cuda:0')
tensor(0., device='cuda:0')
tensor(-10.7117, device='cuda:0')
tensor(0., device='cuda:0')
tensor(-8.4733, device='cuda:0')
tensor(8.8761, device='cuda:0')
tensor(-5.3283, device='cuda:0')
tensor(0., device='cuda:0')
tensor(-12.6979, device='cuda:0')
tensor(0., device='cuda:0')
tensor(-34.9663, device='cuda:0')
tensor(8.5766, device='cuda:0')
tensor(-5.2230, device='cuda:0')
tensor(0., device='cuda:0')
tensor(-10.8700, device='cuda:0')
tensor(7.1779, device='cuda:0')
tensor(-5.3980, device='cuda:0')
tensor(0., device='cuda:0')
tensor(-9.5370, device='cuda:0')
tensor(0., device='cuda:0')
tensor(-27.9718, device='cuda:0')
tensor(0., device='cuda:0')
tensor(-9.4093, device='cuda:0')
tensor(0., device='cuda:0')
tensor(-14.2105, device='cuda:0')
tensor(0., device='cuda:0')
tensor(-9.1506, device='cuda:0')
tensor(0., device='cuda:0')
tensor(-8.8343, device='cuda:0')
t

In [ ]:
prompt = "one two three four"
tokens = model_2.to_tokens(prompt).to(device)

# months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
# for f_ind in list(months_only):
# for f_ind in range(sae_acts_2.shape[-1]):
#     sae_acts_1[0, -1, f_ind] = sae_acts_2[0, -1, f_ind]
# recon = sae.decode(sae_acts_1)
recon = sae.decode(sae_acts_2)
recon.shape

# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch= recon,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

' May'

In [ ]:
prompt = "one two three four"
tokens = model_2.to_tokens(prompt).to(device)

# months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
# for f_ind in list(months_only):
# for f_ind in range(sae_acts_2.shape[-1]):
#     sae_acts_1[0, -1, f_ind] = sae_acts_2[0, -1, f_ind]
# recon = sae.decode(sae_acts_1)
recon = sae.decode(sae_acts_1 - sae_acts_2)
recon.shape

# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch= recon,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

','

In [ ]:
prompt = "one two three four"
tokens = model_2.to_tokens(prompt).to(device)

# months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
# for f_ind in list(months_only):
# for f_ind in range(sae_acts_2.shape[-1]):
#     sae_acts_1[0, -1, f_ind] = sae_acts_2[0, -1, f_ind]
# recon = sae.decode(sae_acts_1)
recon = sae.decode(sae_acts_2 - sae_acts_1)
recon.shape

# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch= recon,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

' April'

In [ ]:
prompt = "one two three four"
tokens = model_2.to_tokens(prompt).to(device)

# months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
# for f_ind in list(months_only):
# for f_ind in range(sae_acts_2.shape[-1]):
#     sae_acts_1[0, -1, f_ind] = sae_acts_2[0, -1, f_ind]
# recon = sae.decode(sae_acts_1)
recon = sae.decode(3*sae_acts_2 - sae_acts_1)
recon.shape

# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch= recon,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

' May'

In [ ]:
prompt = "one two three four"
tokens = model_2.to_tokens(prompt).to(device)

# months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
# for f_ind in list(months_only):
# for f_ind in range(sae_acts_2.shape[-1]):
#     sae_acts_1[0, -1, f_ind] = sae_acts_2[0, -1, f_ind]
# recon = sae.decode(sae_acts_1)
recon = sae.decode(10*sae_acts_1 - sae_acts_2)
recon.shape

# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch= recon,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

'\n'

## steering vec

In [ ]:
prompt = "three four five six"
tokens = model_2.to_tokens(prompt).to(device)

# months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
# for f_ind in list(months_only):
# for f_ind in range(sae_acts_2.shape[-1]):
#     sae_acts_1[0, -1, f_ind] = sae_acts_2[0, -1, f_ind]
# recon = sae.decode(sae_acts_1)

inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_3 = sae.encode(target_act.to(torch.float32))

recon = sae.decode(sae_acts_3)
recon.shape

# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch= recon,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

' seven'

In [ ]:
prompt = "three four five six"
tokens = model_2.to_tokens(prompt).to(device)

# months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
# for f_ind in list(months_only):
# for f_ind in range(sae_acts_2.shape[-1]):
#     sae_acts_1[0, -1, f_ind] = sae_acts_2[0, -1, f_ind]
# recon = sae.decode(sae_acts_1)

inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_3 = sae.encode(target_act.to(torch.float32))

recon = sae.decode(sae_acts_3 + 3*(sae_acts_2 - sae_acts_1))
recon.shape

# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch= recon,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

' April'

In [ ]:
prompt = "two three four five"
tokens = model_2.to_tokens(prompt).to(device)

# months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
# for f_ind in list(months_only):
# for f_ind in range(sae_acts_2.shape[-1]):
#     sae_acts_1[0, -1, f_ind] = sae_acts_2[0, -1, f_ind]
# recon = sae.decode(sae_acts_1)

inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_3 = sae.encode(target_act.to(torch.float32))

recon = sae.decode(sae_acts_3 + 3*(sae_acts_2 - sae_acts_1))
recon.shape

# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch= recon,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

' June'

In [ ]:
prompt = "four five six seven"
tokens = model_2.to_tokens(prompt).to(device)

# months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
# for f_ind in list(months_only):
# for f_ind in range(sae_acts_2.shape[-1]):
#     sae_acts_1[0, -1, f_ind] = sae_acts_2[0, -1, f_ind]
# recon = sae.decode(sae_acts_1)

inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_3 = sae.encode(target_act.to(torch.float32))

recon = sae.decode(sae_acts_3 + 3*(sae_acts_2 - sae_acts_1))
recon.shape

# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch= recon,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

' April'

In [ ]:
prompt = "one two three one"
tokens = model_2.to_tokens(prompt).to(device)

# months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
# for f_ind in list(months_only):
# for f_ind in range(sae_acts_2.shape[-1]):
#     sae_acts_1[0, -1, f_ind] = sae_acts_2[0, -1, f_ind]
# recon = sae.decode(sae_acts_1)

inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_3 = sae.encode(target_act.to(torch.float32))

recon = sae.decode(sae_acts_3 + 3*(sae_acts_2 - sae_acts_1))
recon.shape

# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch= recon,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

' April'

In [ ]:
prompt = "a a a a"
tokens = model_2.to_tokens(prompt).to(device)

# months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
# for f_ind in list(months_only):
# for f_ind in range(sae_acts_2.shape[-1]):
#     sae_acts_1[0, -1, f_ind] = sae_acts_2[0, -1, f_ind]
# recon = sae.decode(sae_acts_1)

inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_3 = sae.encode(target_act.to(torch.float32))

recon = sae.decode(sae_acts_3 + 3*(sae_acts_2 - sae_acts_1))
recon.shape

# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch= recon,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

' April'

In [ ]:
prompt = "a a a a"
tokens = model_2.to_tokens(prompt).to(device)

# months_only = set((two_top_acts_indices).tolist()) - set((one_top_acts_indices).tolist())
# for f_ind in list(months_only):
# for f_ind in range(sae_acts_2.shape[-1]):
#     sae_acts_1[0, -1, f_ind] = sae_acts_2[0, -1, f_ind]
# recon = sae.decode(sae_acts_1)

inputs = tokenizer.encode(prompt, return_tensors="pt", add_special_tokens=True).to("cuda")
target_act = gather_residual_activations(model, 20, inputs)
sae_acts_3 = sae.encode(target_act.to(torch.float32))

recon = sae.decode(3*(sae_acts_2 - sae_acts_1))
recon.shape

# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch= recon,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

' April'

## replace sae_1 with sae_2

In [ ]:
prompt = "one two three four"
tokens = model_2.to_tokens(prompt).to(device)

# replace LLM actvs in that layer with decoder output

from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float, Int

layer_name = 'blocks.20.hook_resid_post'

def patch_layer(
    orig_actvs: Float[Tensor, "batch pos d_model"],
    hook: HookPoint,
    LLM_patch: Float[Tensor, "batch pos d_model"],
    layer_to_patch: int,
) -> Float[Tensor, "batch pos d_model"]:
    if layer_to_patch == hook.layer():
        orig_actvs[:, :, :] = LLM_patch
    return orig_actvs

hook_fn = partial(
        patch_layer,
        LLM_patch= recon,
        layer_to_patch = 20
    )

# if you use run_with_cache, you need to add_hook before
# if you use run_with_hooks, you dont need add_hook, just add it in fwd_hooks arg
# no need to reset hoooks after since run_with_hooks isn't permanent like add_hook with perm arg

# rerun clean inputs on ablated model
ablated_logits = model_2.run_with_hooks(tokens,
                    fwd_hooks=[
                        (layer_name, hook_fn),
                    ]
                )

next_token = ablated_logits[0, -1].argmax(dim=-1)
next_char = model_2.to_string(next_token)
next_char

'\n'